In [ ]:
import nest_asyncio

nest_asyncio.apply()

from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
)
from llama_index.core.evaluation import (
    DatasetGenerator,
    FaithfulnessEvaluator,
    RelevancyEvaluator
)
from llama_index.llms.azure_openai import AzureOpenAI

import openai
import time

# Load environment variables from .env file
import pandas as pd
from llama_index.core.evaluation import (
    RetrieverEvaluator,
    get_retrieval_results_df,
)
import os
import logging
import pandas as pd
from dotenv import load_dotenv
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.readers.file import PyMuPDFReader
from llama_index.core.node_parser import SentenceSplitter
from datetime import datetime
from llama_index.core import (
    StorageContext, VectorStoreIndex, SimpleDirectoryReader, 
    get_response_synthesizer, Settings
)
import traceback
from llama_index.core.evaluation import (
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset,
)
from llama_index.core.evaluation import generate_question_context_pairs, QueryResponseDataset
from llama_index.llms.openai import OpenAI

from typing import List
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore
import asyncio
from tqdm.asyncio import tqdm
from llama_index.core import PromptTemplate
import time
from llama_index.retrievers.bm25 import BM25Retriever


from pathlib import Path
from llama_index.readers.file import PDFReader
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
import json
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import IndexNode
from llama_index.core.embeddings import resolve_embed_model
from llama_index.core import Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import IndexNode
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
)

import giskard
from llama_index.llms.azure_openai import AzureOpenAI
from giskard.llm import set_llm_model, set_llm_api
from giskard.llm.client import get_default_client
from giskard.llm.client import set_llm_api, set_llm_model
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.readers.file import PyMuPDFReader
from llama_index.core.base.llms.types import ChatMessage, MessageRole
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
import os
from giskard.rag import KnowledgeBase, generate_testset, QATestset
#from giskard.rag.question_generators import complex_questions, complex_situational_questions, compare_questions, simple_questions, rule_conclusion_questions, distracting_questions, na_questions, vague_questions, oos_questions, situational_questions, double_questions, conversational_questions
from giskard.rag import AgentAnswer
from giskard.rag import evaluate, RAGReport
from giskard.rag.metrics.ragas_metrics import ragas_context_recall, ragas_faithfulness, ragas_answer_relevancy, ragas_context_precision
from llama_index.core.base.llms.types import ChatMessage, MessageRole
from llama_index.core.query_engine import RetrieverQueryEngine


load_dotenv()
# Fetch API keys from environment variables
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
AZURE_API_KEY = os.getenv('AZURE_API_KEY')
AZURE_DEPLOYMENT_NAME = os.getenv("AZURE_DEPLOYMENT_NAME")
AZURE_API_VERSION = os.getenv("AZURE_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")


embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-large-en-v1.5"
)

Settings.embed_model=embed_model

# setup LLM

llm_gpt4o = AzureOpenAI(
    deployment_name="gpt4o",
    temperature=0, 
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_API_VERSION
)

llm_gpt35 = AzureOpenAI(
    deployment_name="gpt35",#AZURE_DEPLOYMENT_NAME,
    temperature=0, 
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_API_VERSION
)

embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-large-en-v1.5"
)
Settings.embed_model = embed_model

set_llm_api("azure")
set_llm_model('gpt4o')
# You'll need to provide the name of the model that you've deployed
# Beware, the model provided must be capable of using function calls
client = get_default_client()
assert client._client._base_url == f'{os.environ["AZURE_OPENAI_ENDPOINT"]}/openai/'
assert client._client.api_key == os.environ["AZURE_OPENAI_API_KEY"]
assert client._client._api_version == os.environ["OPENAI_API_VERSION"]
assert client.model == "gpt4o"#os.environ["AZURE_OPENAI_CHATGPT_DEPLOYMENT"]

In [ ]:
# Evaluate with Giskard
Settings.llm=llm_gpt35
loader = PyMuPDFReader()
#file_extractor = {".pdf": loader}
documents1 = loader.load(file_path="../legal_data/LL144/LL144.pdf")
documents2 = loader.load(file_path="../legal_data/LL144/LL144_Definitions.pdf")
documents = documents1 + documents2

chunk_size = 512
splitter = SentenceSplitter(chunk_size=chunk_size)
index = VectorStoreIndex.from_documents(documents, llm=llm_gpt35, transformations=[splitter])
chat_engine = index.as_chat_engine()

# Load nodes W/ METADATA REFERENCES INSTEAD
base_nodes = splitter.get_nodes_from_documents(documents)

extractors = [
    SummaryExtractor(summaries=["self"], show_progress=True),
    QuestionsAnsweredExtractor(questions=5, show_progress=True),
]

# run metadata extractor across base nodes, get back dictionaries
node_to_metadata = {}

for extractor in extractors:
    metadata_dicts = extractor.extract(base_nodes)
    for node, metadata in zip(base_nodes, metadata_dicts):
        if node.node_id not in node_to_metadata:
            node_to_metadata[node.node_id] = metadata
        else:
            node_to_metadata[node.node_id].update(metadata)

# cache metadata dicts
def save_metadata_dicts(path, data):
    with open(path, "w") as fp:
        json.dump(data, fp)


def load_metadata_dicts(path):
    with open(path, "r") as fp:
        data = json.load(fp)
    return data

#save_metadata_dicts("metadata_dicts.json", node_to_metadata)
metadata_dicts = load_metadata_dicts("metadata_dicts.json")
metadata = metadata_dicts

# all nodes consists of source nodes, along with metadata
import copy

all_nodes = copy.deepcopy(base_nodes)
for node_id, metadata in node_to_metadata.items():
    for val in metadata.values():
        all_nodes.append(IndexNode(text=val, index_id=node_id))

all_nodes_dict = {n.node_id: n for n in all_nodes}

## Load index into vector index
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI

index = VectorStoreIndex(all_nodes)
chat_engine = index.as_chat_engine()

In [ ]:
print(all_nodes[18])

### WITH TOPICS

In [ ]:
import json
from llama_index.core import PromptTemplate


topics_in_each_node = """
- New York City Department of Consumer and Worker Protection (DCWP)
- Commissioner of the DCWP
- New York City Charter

Bias Audit Requirements
- Clarifications and Definitions
- Public Feedback and Revisions
- Impact Ratio Calculation
- Data for Bias Audits
- Shared Bias Audits

Automated Employment Decision Tool (AEDT)
- Bias Audit
- Candidate for Employment
- Employers
- Employment Agencies
- EEOC
- Independent Auditors
- Bias Audit
- Impact Ratio
- Machine Learning, Statistical Modeling, Data Analytics, or Artificial Intelligence

Scoring Rate
- Screen
- Selection Rate
- Definition of Test Data
- Bias Audit Requirement
- Audit Calculations
- Group Classification
- Exclusion of Unknown Categories
- Example Scenario

Employers
- Employment Agencies
- AEDTs
- Vendors Conducting Bias Audits
- Use of AEDT in Hiring Processes
- Requirement for Bias Audits
- Example of Bias Audit Process
- Metrics for Evaluating Bias (sex categories, number of applicants, number selected, selection rate, impact ratio)

Employer
- AEDT (Automated Employment Decision Tool)
- Vendor
- Independent Auditor
- Applicants (Male and Female)

Race/Ethnicity Categories
- Hispanic or Latino
- White (Not Hispanic or Latino)
- Black or African American (Not Hispanic or Latino)

Intersectional Categories
- Hispanic or Latino
- Non-Hispanic or Latino
- Male

Automated Employment Decision Tool (AEDT)
- Candidates for Employment
- Employees Being Considered for Promotion

Employers
- Independent Auditors
- Demographic Groups (sex, race/ethnicity, intersectional categories)
- Independent Auditor
- Employer
- Automated Employment Decision Tools (AEDTs)

Sex Categories
- Male
- Female

Race/Ethnicity Categories
- Hispanic or Latino
- White (Not Hispanic or Latino)
- Black or African American (Not Hispanic or Latino)
- Native Hawaiian or Pacific Islander (Not Hispanic or Latino)
- Asian (Not Hispanic or Latino)
- Native American or Alaska Native (Not Hispanic or Latino)
- Two or More Races (Not Hispanic or Latino)

Intersectional Categories
- Data Exclusion
- Historical Data Requirements
- Requirements for Conducting a Bias Audit

Automated Employment Decision Tool (AEDT)
- Use of Historical Data and Test Data
- Employment Agencies
- Employers
- AEDTs (Automated Employment Decision Tools)

Bias Audit Restrictions
- Published Results (§ 5-303)
- Notice to Candidates and Employees (§ 5-304)
- Automated Employment Decision Tool (AEDT)
- Employers and Employment Agencies
- Candidates for Employment

Employers
- Employment Agencies
- Candidates for Employment
- New York City Council
- Council Members
- Administrative Code of the City of New York

Bias Audit Requirement
- Notification Requirements
- Legal References
- Disclosure Requirements
- Penalties
- Enforcement

Office of Administrative Trials and Hearings
- City Agencies
- Corporation Counsel
- File #: Int 1894-2020
- New York City Commission on Human Rights
- Title 8
"""

query_gen_prompt_str = (

    "You are an expert legal assistant that generates multiple search queries (e.g. keywords or key phrases) based on a "
    "single input query about a particular set of regulation. Your aim is to gather information"
    "that would contribute to a complete answer to the single query."

    "You have been provided with a list of specific keywords and topics that are discussed in the knowledge "
    "base that the search queries will be applied to. Use these keywords to guide your generation of specific search queries and fill in any gaps."

    "{topics_in_each_node}"

    "Generate {num_queries} search queries, one on each line, "
   "related to the following input query and using the provided keywords appropriately:\n"
    "Query: {query}\n"
    "Queries:\n"
)

num_queries=6
query_str = query_gen_prompt_str
query_gen_prompt = PromptTemplate(query_gen_prompt_str)

fmt_prompt = query_gen_prompt.format(
    num_queries=num_queries - 1, query=query_str, topics_in_each_node=topics_in_each_node
)
print(fmt_prompt)

In [ ]:
from typing import List

from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore
import asyncio

# Reciprocal rank fusion
from typing import List
from llama_index.core.schema import NodeWithScore

# Vector Search on Each QUery

from tqdm.asyncio import tqdm

# Query Gen/Rewriting stage
from llama_index.core import PromptTemplate
import time


import os
from llama_index.llms.groq import Groq
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.azure_openai import AzureOpenAI
from dotenv import load_dotenv

import nest_asyncio

nest_asyncio.apply()

from pathlib import Path
from llama_index.readers.file import PyMuPDFReader

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5")

load_dotenv()
AZURE_API_KEY = os.getenv('AZURE_API_KEY')
AZURE_DEPLOYMENT_NAME = os.getenv("AZURE_DEPLOYMENT_NAME")
AZURE_API_VERSION = os.getenv("AZURE_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")

# setup LLM
import giskard
from llama_index.llms.azure_openai import AzureOpenAI
from giskard.llm import set_llm_model, set_llm_api
from giskard.llm.client import get_default_client
from giskard.llm.client import set_llm_api, set_llm_model
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.readers.file import PyMuPDFReader
from llama_index.core.base.llms.types import ChatMessage, MessageRole
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
import os
from giskard.rag import KnowledgeBase, generate_testset, QATestset


llm_gpt4o = AzureOpenAI(
    deployment_name="gpt4o",
    temperature=0, 
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_API_VERSION
)

llm_gpt35 = AzureOpenAI(
    deployment_name=AZURE_DEPLOYMENT_NAME,
    temperature=0, 
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_API_VERSION
)

embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-large-en-v1.5"
)

Settings.llm = llm_gpt4o
Settings.embed_model = embed_model


from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter


loader = PyMuPDFReader()
documents2 = loader.load(file_path="../legal_data//LL144/LL144_Definitions.pdf")
documents1 = loader.load(file_path="../legal_data//LL144/LL144.pdf")
documents = documents1 + documents2

splitter = SentenceSplitter(chunk_size=512)
#index = VectorStoreIndex.from_documents(
#    documents, transformations=[splitter], embed_model=embed_model, llm=llm_gpt35
#)

index = VectorStoreIndex.from_documents(documents, transformations=[splitter])

query_str = "What is a bias audit and why do I need to do one?"


legal_prompt_str = (
    "You are an expert legal assistant that generates multiple search queries based on a "
    "single input query about a particular set of regulation. Your aim is to gather information"
    "that would contribute to a complete answer to the single query, tailored to legal clients."
    "Consider (if applicable) the following:"
    "- definitions"
    "- requirements"
    "- scope"
    "- jurisdiction"
    "- penalties"
    "Generate {num_queries} search queries, one on each line, "
    "related to the following iput query:\n"
    "Query: {query}\n"
    "Queries:\n"
)

legal_prompt_str = """
You are an expert legal assistant that generates multiple search queries based on a 
single input query about a particular set of regulation. Your aim is to gather information
that would contribute to a complete answer to the single query, tailored to legal clients.

Use any relevant keywords from the following list to improve the specificity of your queries:

- New York City Department of Consumer and Worker Protection (DCWP)
- Commissioner of the DCWP
- New York City Charter

Bias Audit Requirements
- Clarifications and Definitions
- Public Feedback and Revisions
- Impact Ratio Calculation
- Data for Bias Audits
- Shared Bias Audits

Automated Employment Decision Tool (AEDT)
- Bias Audit
- Candidate for Employment
- Employers
- Employment Agencies
- EEOC
- Independent Auditors
- Bias Audit
- Impact Ratio
- Machine Learning, Statistical Modeling, Data Analytics, or Artificial Intelligence

Scoring Rate
- Screen
- Selection Rate
- Definition of Test Data
- Bias Audit Requirement
- Audit Calculations
- Group Classification
- Exclusion of Unknown Categories
- Example Scenario

Employers
- Employment Agencies
- AEDTs
- Vendors Conducting Bias Audits
- Use of AEDT in Hiring Processes
- Requirement for Bias Audits
- Example of Bias Audit Process
- Metrics for Evaluating Bias (sex categories, number of applicants, number selected, selection rate, impact ratio)

Employer
- AEDT (Automated Employment Decision Tool)
- Vendor
- Independent Auditor
- Applicants (Male and Female)

Race/Ethnicity Categories
- Hispanic or Latino
- White (Not Hispanic or Latino)
- Black or African American (Not Hispanic or Latino)

Intersectional Categories
- Hispanic or Latino
- Non-Hispanic or Latino
- Male

Automated Employment Decision Tool (AEDT)
- Candidates for Employment
- Employees Being Considered for Promotion

Employers
- Independent Auditors
- Demographic Groups (sex, race/ethnicity, intersectional categories)
- Independent Auditor
- Employer
- Automated Employment Decision Tools (AEDTs)

Sex Categories
- Male
- Female

Race/Ethnicity Categories
- Hispanic or Latino
- White (Not Hispanic or Latino)
- Black or African American (Not Hispanic or Latino)
- Native Hawaiian or Pacific Islander (Not Hispanic or Latino)
- Asian (Not Hispanic or Latino)
- Native American or Alaska Native (Not Hispanic or Latino)
- Two or More Races (Not Hispanic or Latino)

Intersectional Categories
- Data Exclusion
- Historical Data Requirements
- Requirements for Conducting a Bias Audit

Automated Employment Decision Tool (AEDT)
- Use of Historical Data and Test Data
- Employment Agencies
- Employers
- AEDTs (Automated Employment Decision Tools)

Bias Audit Restrictions
- Published Results (§ 5-303)
- Notice to Candidates and Employees (§ 5-304)
- Automated Employment Decision Tool (AEDT)
- Employers and Employment Agencies
- Candidates for Employment

Employers
- Employment Agencies
- Candidates for Employment
- New York City Council
- Council Members
- Administrative Code of the City of New York

Bias Audit Requirement
- Notification Requirements
- Legal References
- Disclosure Requirements
- Penalties
- Enforcement

Office of Administrative Trials and Hearings
- City Agencies
- Corporation Counsel
- File #: Int 1894-2020
- New York City Commission on Human Rights
- Title 8


Generate {num_queries} search queries, one on each line, 
related to the following input query:\n
Query: {query}\n
Queries:\n


"""

query_gen_prompt_str = legal_prompt_str

results_path = 'rewriter_with_topics4'

query_gen_prompt = PromptTemplate(query_gen_prompt_str)

def generate_queries(llm, query_str: str, num_queries: int = 6):
    fmt_prompt = query_gen_prompt.format(
        num_queries=num_queries - 1, query=query_str
    )
    response = llm.complete(fmt_prompt)
    queries = response.text.split("\n")
    #print(queries)
    return queries


async def run_queries(queries, retrievers):
    """Run queries against retrievers."""
    tasks = []
    for query in queries:
        for i, retriever in enumerate(retrievers):
            tasks.append(retriever.aretrieve(query))

    task_results = await tqdm.gather(*tasks)

    results_dict = {}
    for i, (query, query_result) in enumerate(zip(queries, task_results)):
        results_dict[(query, i)] = query_result

    return results_dict



# get retrievers
from llama_index.retrievers.bm25 import BM25Retriever


## vector retriever
vector_retriever = index.as_retriever(similarity_top_k=5)

## bm25 retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=5
)

def fuse_results(results_dict, similarity_top_k: int = 5):
    """Fuse results."""
    k = 60.0  # `k` is a parameter used to control the impact of outlier rankings.
    fused_scores = {}
    text_to_node = {}

    # compute reciprocal rank scores
    for nodes_with_scores in results_dict.values():
        for rank, node_with_score in enumerate(
            sorted(
                nodes_with_scores, key=lambda x: x.score or 0.0, reverse=True
            )
        ):
            text = node_with_score.node.get_content()
            text_to_node[text] = node_with_score
            if text not in fused_scores:
                fused_scores[text] = 0.0
            fused_scores[text] += 1.0 / (rank + k)

    # sort results
    reranked_results = dict(
        sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    )

    # adjust node scores
    reranked_nodes: List[NodeWithScore] = []
    for text, score in reranked_results.items():
        reranked_nodes.append(text_to_node[text])
        reranked_nodes[-1].score = score

    return reranked_nodes[:similarity_top_k]



class FusionRetriever(BaseRetriever):
    """Ensemble retriever with fusion."""

    def __init__(
        self,
        llm,
        retrievers: List[BaseRetriever],
        similarity_top_k: int = 5,
    ) -> None:
        """Init params."""
        self._retrievers = retrievers
        self._similarity_top_k = similarity_top_k
        self._llm = llm
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        queries = generate_queries(
            self._llm, query_bundle.query_str, num_queries=6
        )
        results = asyncio.run(run_queries(queries, self._retrievers))
        final_results = fuse_results(
            results, similarity_top_k=self._similarity_top_k
        )

        return final_results
    

from llama_index.core.query_engine import RetrieverQueryEngine

splitter = SentenceSplitter(chunk_size=512)

chat_engine = index.as_chat_engine(chat_mode="context")
# create vector index
text_nodes = splitter(documents)
knowledge_base_df = pd.DataFrame([node.text for node in text_nodes], columns=["text"])
knowledge_base = KnowledgeBase(knowledge_base_df)
retriever = FusionRetriever(
    llm_gpt35, [vector_retriever, bm25_retriever], similarity_top_k=5)


Settings.llm = llm_gpt35
def answer_fn(question, history=None):
    if history:
        answer = chat_engine.chat(question, chat_history=[ChatMessage(role=MessageRole.USER if msg["role"] =="user" else MessageRole.ASSISTANT,
                                                          content=msg["content"]) for msg in history])
    else:
        answer = chat_engine.chat(question, chat_history=[])
    return str(answer)

def get_answer_fn(question: str, history=None) -> str:
    """A function representing your RAG agent."""
    messages = history if history else []
    messages.append({"role": "user", "content": question})
    answer = answer_fn(question, history)
    #print(answer)
    retrieved_nodes = retriever.retrieve(question)
    documents = [node.node.text for node in retrieved_nodes]
    # Instead of returning a simple string, we return the AgentAnswer object which
    # allows us to specify the retrieved context which is used by RAGAS metrics
    return AgentAnswer(
        message=answer,
        documents=documents
    )

testset = QATestset.load("../giskard_test_sets/LL144_275_New.jsonl")
report = evaluate(get_answer_fn,
                testset=testset,
                knowledge_base=knowledge_base,
                metrics=[ragas_context_recall, ragas_faithfulness, ragas_answer_relevancy, ragas_context_precision])


results = report.to_pandas()
csv_path = results_path + ".csv"
html_path = results_path + ".html"

results.to_csv(csv_path, index=False)
#report.to_html(html_path)

In [ ]:
results_path = 'rewriter_with_topics_n7'

query_gen_prompt = PromptTemplate(query_gen_prompt_str)

def generate_queries(llm, query_str: str, num_queries: int = 6):
    fmt_prompt = query_gen_prompt.format(
        num_queries=num_queries - 1, query=query_str
    )
    response = llm.complete(fmt_prompt)
    queries = response.text.split("\n")
    #print(queries)
    return queries


async def run_queries(queries, retrievers):
    """Run queries against retrievers."""
    tasks = []
    for query in queries:
        for i, retriever in enumerate(retrievers):
            tasks.append(retriever.aretrieve(query))

    task_results = await tqdm.gather(*tasks)

    results_dict = {}
    for i, (query, query_result) in enumerate(zip(queries, task_results)):
        results_dict[(query, i)] = query_result

    return results_dict

# get retrievers
from llama_index.retrievers.bm25 import BM25Retriever


## vector retriever
vector_retriever = index.as_retriever(similarity_top_k=5)

## bm25 retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=5
)

def fuse_results(results_dict, similarity_top_k: int = 5):
    """Fuse results."""
    k = 60.0  # `k` is a parameter used to control the impact of outlier rankings.
    fused_scores = {}
    text_to_node = {}

    # compute reciprocal rank scores
    for nodes_with_scores in results_dict.values():
        for rank, node_with_score in enumerate(
            sorted(
                nodes_with_scores, key=lambda x: x.score or 0.0, reverse=True
            )
        ):
            text = node_with_score.node.get_content()
            text_to_node[text] = node_with_score
            if text not in fused_scores:
                fused_scores[text] = 0.0
            fused_scores[text] += 1.0 / (rank + k)

    # sort results
    reranked_results = dict(
        sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    )

    # adjust node scores
    reranked_nodes: List[NodeWithScore] = []
    for text, score in reranked_results.items():
        reranked_nodes.append(text_to_node[text])
        reranked_nodes[-1].score = score

    return reranked_nodes[:similarity_top_k]



class FusionRetriever(BaseRetriever):
    """Ensemble retriever with fusion."""

    def __init__(
        self,
        llm,
        retrievers: List[BaseRetriever],
        similarity_top_k: int = 5,
    ) -> None:
        """Init params."""
        self._retrievers = retrievers
        self._similarity_top_k = similarity_top_k
        self._llm = llm
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        queries = generate_queries(
            self._llm, query_bundle.query_str, num_queries=8
        )
        results = asyncio.run(run_queries(queries, self._retrievers))
        final_results = fuse_results(
            results, similarity_top_k=self._similarity_top_k
        )

        return final_results
    

from llama_index.core.query_engine import RetrieverQueryEngine

splitter = SentenceSplitter(chunk_size=512)
#index = VectorStoreIndex.from_documents(documents, transformations=[splitter])
chat_engine = index.as_chat_engine()
# create vector index
text_nodes = splitter(documents)
knowledge_base_df = pd.DataFrame([node.text for node in text_nodes], columns=["text"])
knowledge_base = KnowledgeBase(knowledge_base_df)
retriever = FusionRetriever(
    llm_gpt35, [vector_retriever, bm25_retriever], similarity_top_k=5)


Settings.llm = llm_gpt35
def answer_fn(question, history=None):
    if history:
        answer = chat_engine.chat(question, chat_history=[ChatMessage(role=MessageRole.USER if msg["role"] =="user" else MessageRole.ASSISTANT,
                                                          content=msg["content"]) for msg in history])
    else:
        answer = chat_engine.chat(question, chat_history=[])
    return str(answer)

def get_answer_fn(question: str, history=None) -> str:
    """A function representing your RAG agent."""
    messages = history if history else []
    messages.append({"role": "user", "content": question})
    answer = answer_fn(question, history)
    #print(answer)
    retrieved_nodes = retriever.retrieve(question)
    documents = [node.node.text for node in retrieved_nodes]
    # Instead of returning a simple string, we return the AgentAnswer object which
    # allows us to specify the retrieved context which is used by RAGAS metrics
    return AgentAnswer(
        message=answer,
        documents=documents
    )

testset = QATestset.load("../giskard_test_sets/LL144_275_New.jsonl")
report = evaluate(get_answer_fn,
                testset=testset,
                knowledge_base=knowledge_base,
                metrics=[ragas_context_recall, ragas_faithfulness, ragas_answer_relevancy, ragas_context_precision])


results = report.to_pandas()
csv_path = results_path + ".csv"
html_path = results_path + ".html"

results.to_csv(csv_path, index=False)
report.to_html(html_path)

In [ ]:
results_path = 'rewriter_with_topics_n10'

query_gen_prompt = PromptTemplate(query_gen_prompt_str)

def generate_queries(llm, query_str: str, num_queries: int = 6):
    fmt_prompt = query_gen_prompt.format(
        num_queries=num_queries - 1, query=query_str
    )
    response = llm.complete(fmt_prompt)
    queries = response.text.split("\n")
    #print(queries)
    return queries


async def run_queries(queries, retrievers):
    """Run queries against retrievers."""
    tasks = []
    for query in queries:
        for i, retriever in enumerate(retrievers):
            tasks.append(retriever.aretrieve(query))

    task_results = await tqdm.gather(*tasks)

    results_dict = {}
    for i, (query, query_result) in enumerate(zip(queries, task_results)):
        results_dict[(query, i)] = query_result

    return results_dict



# get retrievers
from llama_index.retrievers.bm25 import BM25Retriever


## vector retriever
vector_retriever = index.as_retriever(similarity_top_k=5)

## bm25 retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=5
)

def fuse_results(results_dict, similarity_top_k: int = 5):
    """Fuse results."""
    k = 60.0  # `k` is a parameter used to control the impact of outlier rankings.
    fused_scores = {}
    text_to_node = {}

    # compute reciprocal rank scores
    for nodes_with_scores in results_dict.values():
        for rank, node_with_score in enumerate(
            sorted(
                nodes_with_scores, key=lambda x: x.score or 0.0, reverse=True
            )
        ):
            text = node_with_score.node.get_content()
            text_to_node[text] = node_with_score
            if text not in fused_scores:
                fused_scores[text] = 0.0
            fused_scores[text] += 1.0 / (rank + k)

    # sort results
    reranked_results = dict(
        sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    )

    # adjust node scores
    reranked_nodes: List[NodeWithScore] = []
    for text, score in reranked_results.items():
        reranked_nodes.append(text_to_node[text])
        reranked_nodes[-1].score = score

    return reranked_nodes[:similarity_top_k]



class FusionRetriever(BaseRetriever):
    """Ensemble retriever with fusion."""

    def __init__(
        self,
        llm,
        retrievers: List[BaseRetriever],
        similarity_top_k: int = 5,
    ) -> None:
        """Init params."""
        self._retrievers = retrievers
        self._similarity_top_k = similarity_top_k
        self._llm = llm
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        queries = generate_queries(
            self._llm, query_bundle.query_str, num_queries=11
        )
        results = asyncio.run(run_queries(queries, self._retrievers))
        final_results = fuse_results(
            results, similarity_top_k=self._similarity_top_k
        )

        return final_results
    

from llama_index.core.query_engine import RetrieverQueryEngine

splitter = SentenceSplitter(chunk_size=512)
#index = VectorStoreIndex.from_documents(documents, transformations=[splitter])
chat_engine = index.as_chat_engine()
# create vector index
text_nodes = splitter(documents)
knowledge_base_df = pd.DataFrame([node.text for node in text_nodes], columns=["text"])
knowledge_base = KnowledgeBase(knowledge_base_df)
retriever = FusionRetriever(
    llm_gpt35, [vector_retriever, bm25_retriever], similarity_top_k=5)


import tiktoken

Settings.llm = llm_gpt35
def answer_fn(question, history=None):
    if history:
        answer = chat_engine.chat(question, chat_history=[ChatMessage(role=MessageRole.USER if msg["role"] =="user" else MessageRole.ASSISTANT,
                                                          content=msg["content"]) for msg in history])
    else:
        answer = chat_engine.chat(question, chat_history=[])
    return str(answer)

def get_answer_fn(question: str, history=None) -> str:
    """A function representing your RAG agent."""
    messages = history if history else []
    messages.append({"role": "user", "content": question})
    answer = answer_fn(question, history)
    #encoding = tiktoken.get_encoding('cl100k_base')

    #print(answer)
    retrieved_nodes = retriever.retrieve(question)
    documents = [node.node.text for node in retrieved_nodes]

    #docs_tokens = encoding.encode(documents)
    # Instead of returning a simple string, we return the AgentAnswer object which
    # allows us to specify the retrieved context which is used by RAGAS metrics
    return AgentAnswer(
        message=answer,
        documents=documents
    )

testset = QATestset.load("../giskard_test_sets/LL144_275_New.jsonl")
report = evaluate(get_answer_fn,
                testset=testset,
                knowledge_base=knowledge_base,
                metrics=[ragas_context_recall, ragas_faithfulness, ragas_answer_relevancy, ragas_context_precision])


results = report.to_pandas()
csv_path = results_path + ".csv"
html_path = results_path + ".html"

results.to_csv(csv_path, index=False)
report.to_html(html_path)

#### No Rewriter, Just Vary K



In [ ]:
from typing import List
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore
import asyncio
from tqdm.asyncio import tqdm
import os
from dotenv import load_dotenv
import nest_asyncio
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, Settings
from llama_index.core.node_parser import SentenceSplitter
import pandas as pd
from giskard.rag import KnowledgeBase, QATestset, generate_testset
from giskard import evaluate
from giskard.llm.client import set_llm_model, set_llm_api, get_default_client
from giskard.llm import set_llm_model
from llama_index.core.base.llms.types import ChatMessage, MessageRole

nest_asyncio.apply()

# Load environment variables
load_dotenv()
AZURE_API_KEY = os.getenv('AZURE_API_KEY')
AZURE_DEPLOYMENT_NAME = os.getenv("AZURE_DEPLOYMENT_NAME")
AZURE_API_VERSION = os.getenv("AZURE_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")

# Setup LLM
llm_gpt4o = AzureOpenAI(
    deployment_name="gpt4o",
    temperature=0, 
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_API_VERSION
)

llm_gpt35 = AzureOpenAI(
    deployment_name=AZURE_DEPLOYMENT_NAME,
    temperature=0, 
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_API_VERSION
)

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5")

Settings.llm = llm_gpt4o
Settings.embed_model = embed_model

# Load documents
loader = PyMuPDFReader()
documents2 = loader.load(file_path="../legal_data//LL144/LL144_Definitions.pdf")
documents1 = loader.load(file_path="../legal_data//LL144/LL144.pdf")
documents = documents1 + documents2

splitter = SentenceSplitter(chunk_size=512)

# Define the query
query_str = "What is a bias audit and why do I need to do one?"

async def run_queries(queries, retrievers):
    """Run queries against retrievers."""
    tasks = []
    for query in queries:
        for i, retriever in enumerate(retrievers):
            tasks.append(retriever.aretrieve(query))

    task_results = await tqdm.gather(*tasks)

    results_dict = {}
    for i, (query, query_result) in enumerate(zip(queries, task_results)):
        results_dict[(query, i)] = query_result

    return results_dict

# Get retrievers
from llama_index.retrievers.bm25 import BM25Retriever

## vector retriever
vector_retriever = index.as_retriever(similarity_top_k=5)

## bm25 retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=5
)

def fuse_results(results_dict, similarity_top_k: int = 5):
    """Fuse results."""
    k = 60.0  # `k` is a parameter used to control the impact of outlier rankings.
    fused_scores = {}
    text_to_node = {}

    # compute reciprocal rank scores
    for nodes_with_scores in results_dict.values():
        for rank, node_with_score in enumerate(
            sorted(
                nodes_with_scores, key=lambda x: x.score or 0.0, reverse=True
            )
        ):
            text = node_with_score.node.get_content()
            text_to_node[text] = node_with_score
            if text not in fused_scores:
                fused_scores[text] = 0.0
            fused_scores[text] += 1.0 / (rank + k)

    # sort results
    reranked_results = dict(
        sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    )

    # adjust node scores
    reranked_nodes: List[NodeWithScore] = []
    for text, score in reranked_results.items():
        reranked_nodes.append(text_to_node[text])
        reranked_nodes[-1].score = score

    return reranked_nodes[:similarity_top_k]

class FusionRetriever(BaseRetriever):
    """Ensemble retriever with fusion."""

    def __init__(
        self,
        llm,
        retrievers: List[BaseRetriever],
        similarity_top_k: int = 5,
    ) -> None:
        """Init params."""
        self._retrievers = retrievers
        self._similarity_top_k = similarity_top_k
        self._llm = llm
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        queries = [query_bundle.query_str]  # Pass through the original query directly
        results = asyncio.run(run_queries(queries, self._retrievers))
        final_results = fuse_results(
            results, similarity_top_k=self._similarity_top_k
        )

        return final_results

from llama_index.core.query_engine import RetrieverQueryEngine

splitter = SentenceSplitter(chunk_size=512)
#index = VectorStoreIndex.from_documents(documents, transformations=[splitter])
chat_engine = index.as_chat_engine()
# create vector index
text_nodes = splitter(documents)
knowledge_base_df = pd.DataFrame([node.text for node in text_nodes], columns=["text"])
knowledge_base = KnowledgeBase(knowledge_base_df)
retriever = FusionRetriever(
    llm_gpt35, [vector_retriever, bm25_retriever], similarity_top_k=5)

Settings.llm = llm_gpt35

def answer_fn(question, history=None):
    if history:
        answer = chat_engine.chat(question, chat_history=[ChatMessage(role=MessageRole.USER if msg["role"] =="user" else MessageRole.ASSISTANT,
                                                          content=msg["content"]) for msg in history])
    else:
        answer = chat_engine.chat(question, chat_history=[])
    return str(answer)

def get_answer_fn(question: str, history=None) -> str:
    """A function representing your RAG agent."""
    messages = history if history else []
    messages.append({"role": "user", "content": question})
    answer = answer_fn(question, history)
    print(answer)
    retrieved_nodes = retriever.retrieve(question)
    documents = [node.node.text for node in retrieved_nodes]
    # Instead of returning a simple string, we return the AgentAnswer object which
    # allows us to specify the retrieved context which is used by RAGAS metrics
    return AgentAnswer(
        message=answer,
        documents=documents
    )

testset = QATestset.load("../giskard_test_sets/LL144_275_New.jsonl")
report = evaluate(get_answer_fn,
                testset=testset,
                knowledge_base=knowledge_base,
                metrics=[ragas_context_recall, ragas_faithfulness, ragas_answer_relevancy, ragas_context_precision])

results_path = 'results_no_rewriter'

results = report.to_pandas()
csv_path = results_path + ".csv"
html_path = results_path + ".html"

results.to_csv(csv_path, index=False)
#report.to_html(html_path)


### NO TOPICS

In [ ]:
query_gen_prompt_str = (

    "You are an expert legal assistant that generates multiple search queries (e.g. keywords or key phrases) based on a "
    "single input query. Your aim is to generate search queries to gather information"
    "that would contribute to a complete answer to the original single query."

    "Generate {num_queries} search queries, one on each line, "
   "related to the following input query:\n"
    "Query: {query}\n"
    "Queries:\n"
)

topics_in_each_node=None

num_queries=6
query_str = query_gen_prompt_str
query_gen_prompt = PromptTemplate(query_gen_prompt_str)

fmt_prompt = query_gen_prompt.format(
    num_queries=num_queries - 1, query=query_str, topics_in_each_node=topics_in_each_node
)
print(fmt_prompt)

### Evaluate Rewriter w/ topics


In [ ]:
from typing import List
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore
import asyncio
from tqdm.asyncio import tqdm
import os
from dotenv import load_dotenv
import nest_asyncio
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, Settings
from llama_index.core.node_parser import SentenceSplitter
import pandas as pd
from giskard.rag import KnowledgeBase, QATestset, generate_testset
from giskard.llm.client import set_llm_model, set_llm_api, get_default_client
from giskard.llm import set_llm_model
from llama_index.core.base.llms.types import ChatMessage, MessageRole

nest_asyncio.apply()

# Load environment variables
load_dotenv()
AZURE_API_KEY = os.getenv('AZURE_API_KEY')
AZURE_DEPLOYMENT_NAME = os.getenv("AZURE_DEPLOYMENT_NAME")
AZURE_API_VERSION = os.getenv("AZURE_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")

# Setup LLM
llm_gpt4o = AzureOpenAI(
    deployment_name="gpt4o",
    temperature=0, 
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_API_VERSION
)

llm_gpt35 = AzureOpenAI(
    deployment_name=AZURE_DEPLOYMENT_NAME,
    temperature=0, 
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_API_VERSION
)

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5")

Settings.llm = llm_gpt4o
Settings.embed_model = embed_model

# Load documents
loader = PyMuPDFReader()
documents2 = loader.load(file_path="../legal_data//LL144/LL144_Definitions.pdf")
documents1 = loader.load(file_path="../legal_data//LL144/LL144.pdf")
documents = documents1 + documents2

splitter = SentenceSplitter(chunk_size=512)

# Define the query
query_str = "What is a bias audit and why do I need to do one?"

async def run_queries(queries, retrievers):
    """Run queries against retrievers."""
    tasks = []
    for query in queries:
        for i, retriever in enumerate(retrievers):
            tasks.append(retriever.aretrieve(query))

    task_results = await tqdm.gather(*tasks)

    results_dict = {}
    for i, (query, query_result) in enumerate(zip(queries, task_results)):
        results_dict[(query, i)] = query_result

    return results_dict

# Get retrievers
from llama_index.retrievers.bm25 import BM25Retriever

## vector retriever
vector_retriever = index.as_retriever(similarity_top_k=10)

## bm25 retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=10
)

def fuse_results(results_dict, similarity_top_k: int = 5):
    """Fuse results."""
    k = 60.0  # `k` is a parameter used to control the impact of outlier rankings.
    fused_scores = {}
    text_to_node = {}

    # compute reciprocal rank scores
    for nodes_with_scores in results_dict.values():
        for rank, node_with_score in enumerate(
            sorted(
                nodes_with_scores, key=lambda x: x.score or 0.0, reverse=True
            )
        ):
            text = node_with_score.node.get_content()
            text_to_node[text] = node_with_score
            if text not in fused_scores:
                fused_scores[text] = 0.0
            fused_scores[text] += 1.0 / (rank + k)

    # sort results
    reranked_results = dict(
        sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    )

    # adjust node scores
    reranked_nodes: List[NodeWithScore] = []
    for text, score in reranked_results.items():
        reranked_nodes.append(text_to_node[text])
        reranked_nodes[-1].score = score

    return reranked_nodes[:similarity_top_k]

class FusionRetriever(BaseRetriever):
    """Ensemble retriever with fusion."""

    def __init__(
        self,
        llm,
        retrievers: List[BaseRetriever],
        similarity_top_k: int = 5,
    ) -> None:
        """Init params."""
        self._retrievers = retrievers
        self._similarity_top_k = similarity_top_k
        self._llm = llm
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        queries = [query_bundle.query_str]  # Pass through the original query directly
        results = asyncio.run(run_queries(queries, self._retrievers))
        final_results = fuse_results(
            results, similarity_top_k=self._similarity_top_k
        )

        return final_results

from llama_index.core.query_engine import RetrieverQueryEngine

splitter = SentenceSplitter(chunk_size=512)
#index = VectorStoreIndex.from_documents(documents, transformations=[splitter])
chat_engine = index.as_chat_engine()
# create vector index
text_nodes = splitter(documents)
knowledge_base_df = pd.DataFrame([node.text for node in text_nodes], columns=["text"])
knowledge_base = KnowledgeBase(knowledge_base_df)
retriever = FusionRetriever(
    llm_gpt35, [vector_retriever, bm25_retriever], similarity_top_k=5)

Settings.llm = llm_gpt35

def answer_fn(question, history=None):
    if history:
        answer = chat_engine.chat(question, chat_history=[ChatMessage(role=MessageRole.USER if msg["role"] =="user" else MessageRole.ASSISTANT,
                                                          content=msg["content"]) for msg in history])
    else:
        answer = chat_engine.chat(question, chat_history=[])
    return str(answer)

def get_answer_fn(question: str, history=None) -> str:
    """A function representing your RAG agent."""
    messages = history if history else []
    messages.append({"role": "user", "content": question})
    answer = answer_fn(question, history)
    #print(answer)
    retrieved_nodes = retriever.retrieve(question)
    documents = [node.node.text for node in retrieved_nodes]
    # Instead of returning a simple string, we return the AgentAnswer object which
    # allows us to specify the retrieved context which is used by RAGAS metrics
    return AgentAnswer(
        message=answer,
        documents=documents
    )

testset = QATestset.load("../giskard_test_sets/LL144_275_New.jsonl")
report = evaluate(get_answer_fn,
                testset=testset,
                knowledge_base=knowledge_base,
                metrics=[ragas_context_recall, ragas_faithfulness, ragas_answer_relevancy, ragas_context_precision])

results_path = 'results_no_rewriter_k5'

results = report.to_pandas()
csv_path = results_path + ".csv"
html_path = results_path + ".html"

results.to_csv(csv_path, index=False)
#report.to_html(html_path)


In [ ]:
from typing import List
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore
import asyncio
from tqdm.asyncio import tqdm
import os
from dotenv import load_dotenv
import nest_asyncio
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, Settings
from llama_index.core.node_parser import SentenceSplitter
import pandas as pd
from giskard.rag import KnowledgeBase, QATestset, generate_testset
from giskard.llm.client import set_llm_model, set_llm_api, get_default_client
from giskard.llm import set_llm_model
from llama_index.core.base.llms.types import ChatMessage, MessageRole

nest_asyncio.apply()

# Load environment variables
load_dotenv()
AZURE_API_KEY = os.getenv('AZURE_API_KEY')
AZURE_DEPLOYMENT_NAME = os.getenv("AZURE_DEPLOYMENT_NAME")
AZURE_API_VERSION = os.getenv("AZURE_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")

# Setup LLM
llm_gpt4o = AzureOpenAI(
    deployment_name="gpt4o",
    temperature=0, 
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_API_VERSION
)

llm_gpt35 = AzureOpenAI(
    deployment_name=AZURE_DEPLOYMENT_NAME,
    temperature=0, 
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_API_VERSION
)

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5")

Settings.llm = llm_gpt4o
Settings.embed_model = embed_model

# Load documents
loader = PyMuPDFReader()
documents2 = loader.load(file_path="../legal_data//LL144/LL144_Definitions.pdf")
documents1 = loader.load(file_path="../legal_data//LL144/LL144.pdf")
documents = documents1 + documents2

splitter = SentenceSplitter(chunk_size=512)

# Define the query
query_str = "What is a bias audit and why do I need to do one?"

async def run_queries(queries, retrievers):
    """Run queries against retrievers."""
    tasks = []
    for query in queries:
        for i, retriever in enumerate(retrievers):
            tasks.append(retriever.aretrieve(query))

    task_results = await tqdm.gather(*tasks)

    results_dict = {}
    for i, (query, query_result) in enumerate(zip(queries, task_results)):
        results_dict[(query, i)] = query_result

    return results_dict

# Get retrievers
from llama_index.retrievers.bm25 import BM25Retriever

## vector retriever
vector_retriever = index.as_retriever(similarity_top_k=10)

## bm25 retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=10
)

def fuse_results(results_dict, similarity_top_k: int = 5):
    """Fuse results."""
    k = 60.0  # `k` is a parameter used to control the impact of outlier rankings.
    fused_scores = {}
    text_to_node = {}

    # compute reciprocal rank scores
    for nodes_with_scores in results_dict.values():
        for rank, node_with_score in enumerate(
            sorted(
                nodes_with_scores, key=lambda x: x.score or 0.0, reverse=True
            )
        ):
            text = node_with_score.node.get_content()
            text_to_node[text] = node_with_score
            if text not in fused_scores:
                fused_scores[text] = 0.0
            fused_scores[text] += 1.0 / (rank + k)

    # sort results
    reranked_results = dict(
        sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    )

    # adjust node scores
    reranked_nodes: List[NodeWithScore] = []
    for text, score in reranked_results.items():
        reranked_nodes.append(text_to_node[text])
        reranked_nodes[-1].score = score

    return reranked_nodes[:similarity_top_k]

class FusionRetriever(BaseRetriever):
    """Ensemble retriever with fusion."""

    def __init__(
        self,
        llm,
        retrievers: List[BaseRetriever],
        similarity_top_k: int = 5,
    ) -> None:
        """Init params."""
        self._retrievers = retrievers
        self._similarity_top_k = similarity_top_k
        self._llm = llm
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        queries = [query_bundle.query_str]  # Pass through the original query directly
        results = asyncio.run(run_queries(queries, self._retrievers))
        final_results = fuse_results(
            results, similarity_top_k=self._similarity_top_k
        )

        return final_results

from llama_index.core.query_engine import RetrieverQueryEngine

splitter = SentenceSplitter(chunk_size=512)
#index = VectorStoreIndex.from_documents(documents, transformations=[splitter])
chat_engine = index.as_chat_engine()
# create vector index
text_nodes = splitter(documents)
knowledge_base_df = pd.DataFrame([node.text for node in text_nodes], columns=["text"])
knowledge_base = KnowledgeBase(knowledge_base_df)
retriever = FusionRetriever(
    llm_gpt35, [vector_retriever, bm25_retriever], similarity_top_k=7)

Settings.llm = llm_gpt35

def answer_fn(question, history=None):
    if history:
        answer = chat_engine.chat(question, chat_history=[ChatMessage(role=MessageRole.USER if msg["role"] =="user" else MessageRole.ASSISTANT,
                                                          content=msg["content"]) for msg in history])
    else:
        answer = chat_engine.chat(question, chat_history=[])
    return str(answer)

def get_answer_fn(question: str, history=None) -> str:
    """A function representing your RAG agent."""
    messages = history if history else []
    messages.append({"role": "user", "content": question})
    answer = answer_fn(question, history)
    #print(answer)
    retrieved_nodes = retriever.retrieve(question)
    documents = [node.node.text for node in retrieved_nodes]
    # Instead of returning a simple string, we return the AgentAnswer object which
    # allows us to specify the retrieved context which is used by RAGAS metrics
    return AgentAnswer(
        message=answer,
        documents=documents
    )

testset = QATestset.load("../giskard_test_sets/LL144_275_New.jsonl")
"""report = evaluate(get_answer_fn,
                testset=testset,
                knowledge_base=knowledge_base,
                metrics=[ragas_context_recall, ragas_faithfulness, ragas_answer_relevancy, ragas_context_precision])

results_path = 'results_no_rewriter_k7'

results = report.to_pandas()
csv_path = results_path + ".csv"
html_path = results_path + ".html"

results.to_csv(csv_path, index=False)
#report.to_html(html_path)"""


In [ ]:
run_nodes = text_nodes

for idx, node in enumerate(run_nodes):
    node.id_ = f"node_{idx}"

vector_index = VectorStoreIndex(run_nodes)

## vector retriever
vector_retriever = vector_index.as_retriever(similarity_top_k=10)

## bm25 retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=vector_index.docstore, similarity_top_k=10
)


retriever = FusionRetriever(llm_gpt35, [vector_retriever, bm25_retriever], similarity_top_k=10)

retrieved_nodes = retriever.retrieve("What is a bias audit?")

from llama_index.core.response.notebook_utils import display_source_node

for node in retrieved_nodes:
    display_source_node(node, source_length=1000)

In [ ]:
# [optional] load
from llama_index.core.evaluation import RetrieverEvaluator
from llama_index.core.evaluation import (
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset,
)
qa_dataset = EmbeddingQAFinetuneDataset.from_json("../retriever_eval_sets/retriever_eval_dataset.json")
metrics = ["hit_rate", "mrr", "precision", "recall", "ap", "ndcg"]
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    metrics, retriever=retriever
)
# try it out on a sample query
sample_id, sample_query = list(qa_dataset.queries.items())[0]
sample_expected = qa_dataset.relevant_docs[sample_id]

eval_result = retriever_evaluator.evaluate(sample_query, sample_expected)
print(eval_result)

In [ ]:
# Load the documents using PyMuPDFReader
loader = PyMuPDFReader()
docs1 = loader.load(file_path="../legal_data/LL144/LL144.pdf")
docs2 = loader.load(file_path="../legal_data/LL144/LL144_Definitions.pdf")

# Combine the documents
docs = docs1 + docs2

# Initialize an empty string to store all text
all_text = ""

# Iterate through the documents and concatenate the text
for doc in docs:
    all_text += doc.text  # Assuming 'text' attribute contains the document text

# Define the token to split on
token = "\n§"

# Create a regular expression pattern with a lookahead assertion to split before the token
pattern = re.compile(r"(?=" + re.escape(token) + ")")

# Split the text based on the pattern
split_text = pattern.split(all_text)

# Create TextNode objects from the split text
text_nodes = []
for idx, part in enumerate(split_text):
    node = TextNode(text=part)
    text_nodes.append(node)

# Print the number of TextNode objects created to verify
print(f"Number of TextNode objects created: {len(text_nodes)}")

# Create the VectorStoreIndex with your own TextNode objects
index = VectorStoreIndex(text_nodes)
chat_engine = index.as_chat_engine(chat_mode="context")

response = chat_engine.chat("what a bias audit")
print(f"Chat Engine Response: {response}")


In [ ]:

# Load environment variables
load_dotenv()
AZURE_API_KEY = os.getenv('AZURE_API_KEY')
AZURE_DEPLOYMENT_NAME = os.getenv("AZURE_DEPLOYMENT_NAME")
AZURE_API_VERSION = os.getenv("AZURE_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")

# Setup LLM
llm_gpt4o = AzureOpenAI(
    deployment_name="gpt4o",
    temperature=0, 
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_API_VERSION
)

llm_gpt35 = AzureOpenAI(
    deployment_name=AZURE_DEPLOYMENT_NAME,
    temperature=0, 
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_API_VERSION
)

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5")

Settings.llm = llm_gpt35
Settings.embed_model = embed_model


# Define the query
query_str = "What is a bias audit and why do I need to do one?"

async def run_queries(queries, retrievers):
    """Run queries against retrievers."""
    tasks = []
    for query in queries:
        for i, retriever in enumerate(retrievers):
            tasks.append(retriever.aretrieve(query))

    task_results = await tqdm.gather(*tasks)

    results_dict = {}
    for i, (query, query_result) in enumerate(zip(queries, task_results)):
        results_dict[(query, i)] = query_result

    return results_dict

# Get retrievers
from llama_index.retrievers.bm25 import BM25Retriever

## vector retriever
vector_retriever = index.as_retriever(similarity_top_k=10)

## bm25 retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=10
)

def fuse_results(results_dict, similarity_top_k: int = 5):
    """Fuse results."""
    k = 60.0  # `k` is a parameter used to control the impact of outlier rankings.
    fused_scores = {}
    text_to_node = {}

    # compute reciprocal rank scores
    for nodes_with_scores in results_dict.values():
        for rank, node_with_score in enumerate(
            sorted(
                nodes_with_scores, key=lambda x: x.score or 0.0, reverse=True
            )
        ):
            text = node_with_score.node.get_content()
            text_to_node[text] = node_with_score
            if text not in fused_scores:
                fused_scores[text] = 0.0
            fused_scores[text] += 1.0 / (rank + k)

    # sort results
    reranked_results = dict(
        sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    )

    # adjust node scores
    reranked_nodes: List[NodeWithScore] = []
    for text, score in reranked_results.items():
        reranked_nodes.append(text_to_node[text])
        reranked_nodes[-1].score = score

    return reranked_nodes[:similarity_top_k]

class FusionRetriever(BaseRetriever):
    """Ensemble retriever with fusion."""

    def __init__(
        self,
        llm,
        retrievers: List[BaseRetriever],
        similarity_top_k: int = 5,
    ) -> None:
        """Init params."""
        self._retrievers = retrievers
        self._similarity_top_k = similarity_top_k
        self._llm = llm
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        queries = [query_bundle.query_str]  # Pass through the original query directly
        results = asyncio.run(run_queries(queries, self._retrievers))
        final_results = fuse_results(
            results, similarity_top_k=self._similarity_top_k
        )

        return final_results

from llama_index.core.query_engine import RetrieverQueryEngine

knowledge_base_df = pd.DataFrame([node.text for node in text_nodes], columns=["text"])
knowledge_base = KnowledgeBase(knowledge_base_df)
retriever = FusionRetriever(
    llm_gpt35, [vector_retriever, bm25_retriever], similarity_top_k=5)

Settings.llm = llm_gpt35

def answer_fn(question, history=None):
    if history:
        answer = chat_engine.chat(question, chat_history=[ChatMessage(role=MessageRole.USER if msg["role"] =="user" else MessageRole.ASSISTANT,
                                                          content=msg["content"]) for msg in history])
    else:
        answer = chat_engine.chat(question, chat_history=[])
    return str(answer)

def get_answer_fn(question: str, history=None) -> str:
    """A function representing your RAG agent."""
    messages = history if history else []
    messages.append({"role": "user", "content": question})
    answer = answer_fn(question, history)
    #print(answer)
    retrieved_nodes = retriever.retrieve(question)
    documents = [node.node.text for node in retrieved_nodes]
    # Instead of returning a simple string, we return the AgentAnswer object which
    # allows us to specify the retrieved context which is used by RAGAS metrics
    return AgentAnswer(
        message=answer,
        documents=documents
    )

testset = QATestset.load("../giskard_test_sets/LL144_275_New.jsonl")
report = evaluate(get_answer_fn,
                testset=testset,
                knowledge_base=knowledge_base,
                metrics=[ragas_context_recall, ragas_faithfulness, ragas_answer_relevancy, ragas_context_precision])

results_path = 'char_split_results2'

results = report.to_pandas()
csv_path = results_path + ".csv"
html_path = results_path + ".html"

results.to_csv(csv_path, index=False)
#report.to_html(html_path)



In [ ]:
from typing import List
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore
import asyncio
from tqdm.asyncio import tqdm
from llama_index.core import PromptTemplate
import time
from llama_index.retrievers.bm25 import BM25Retriever



#query_str = "What is a bias audit and why do I need to do one?"

#query_gen_prompt_str = (
#    "You are an expert legal assistant that generates multiple search queries based on a "
#    "single input query about a particular set of regulation. Your aim is to gather information"
#    "that would contribute to a complete answer to the single query, tailored to legal clients."
#    "Consider (if applicable) the following:"
#    "- definitions"
#    "- requirements"
#    "- scope"
#    "- jurisdiction"
#    "- penalties"
#    "Generate {num_queries} search queries, one on each line, "
#    "related to the following iput query:\n"
#    "Query: {query}\n"
#    "Queries:\n"
#)
import time


Settings.llm=llm_gpt35
query_gen_prompt = PromptTemplate(query_gen_prompt_str)

def generate_queries(llm, query_str: str, num_queries: int = 6, topics_in_each_node=topics_in_each_node):
    fmt_prompt = query_gen_prompt.format(
        num_queries=num_queries - 1, query=query_str, topics_in_each_node=topics_in_each_node
    )
    #print(fmt_prompt)
    #time.sleep(2)
    response = llm.complete(fmt_prompt)
    #time.sleep(2)
    queries = response.text.split("\n")
    print(queries)
    return queries

async def run_queries(queries, retrievers):
    """Run queries against retrievers."""
    tasks = []
    for query in queries:
        for i, retriever in enumerate(retrievers):
            tasks.append(retriever.aretrieve(query))

    task_results = await tqdm.gather(*tasks)

    results_dict = {}
    for i, (query, query_result) in enumerate(zip(queries, task_results)):
        results_dict[(query, i)] = query_result
    #print(results_dict)
    return results_dict

## vector retriever
vector_retriever = index.as_retriever(similarity_top_k=10)

## bm25 retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=10
)

def fuse_results(results_dict, similarity_top_k: int = 5):
    """Fuse results."""
    k = 60.0  # `k` is a parameter used to control the impact of outlier rankings.
    fused_scores = {}
    text_to_node = {}

    # compute reciprocal rank scores
    for nodes_with_scores in results_dict.values():
        for rank, node_with_score in enumerate(
            sorted(
                nodes_with_scores, key=lambda x: x.score or 0.0, reverse=True
            )
        ):
            text = node_with_score.node.get_content()
            text_to_node[text] = node_with_score
            if text not in fused_scores:
                fused_scores[text] = 0.0
            fused_scores[text] += 1.0 / (rank + k)

    # sort results
    reranked_results = dict(
        sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    )

    # adjust node scores
    reranked_nodes: List[NodeWithScore] = []
    for text, score in reranked_results.items():
        reranked_nodes.append(text_to_node[text])
        reranked_nodes[-1].score = score

    return reranked_nodes[:similarity_top_k]


class FusionRetriever(BaseRetriever):
    """Ensemble retriever with fusion."""

    def __init__(
        self,
        llm,
        retrievers: List[BaseRetriever],
        similarity_top_k: int = 5,
    ) -> None:
        """Init params."""
        self._retrievers = retrievers
        self._similarity_top_k = similarity_top_k
        self._llm = llm
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        queries = generate_queries(
            self._llm, query_bundle.query_str, num_queries=6, topics_in_each_node=topics_in_each_node
        )

        results = asyncio.run(run_queries(queries, self._retrievers))

        final_results = fuse_results(
            results, similarity_top_k=self._similarity_top_k
        )
        #print(f"final_results {final_results}")

        return final_results
    

from llama_index.core.query_engine import RetrieverQueryEngine

#fusion_retriever = FusionRetriever(
 #   llm_gpt35, [vector_retriever, bm25_retriever], similarity_top_k=3
#)

# create vector index
text_nodes = splitter(documents)
knowledge_base_df = pd.DataFrame([node.text for node in all_nodes], columns=["text"])
knowledge_base = KnowledgeBase(knowledge_base_df)
retriever = FusionRetriever(
    llm_gpt35, [vector_retriever, bm25_retriever], similarity_top_k=5)#index.as_retriever(similarity_top_k=5)

#retriever = create_adaptive_retriever(index=index, llm=llm_gpt35, similarity_top_k=10)


Settings.llm = llm_gpt4o
def answer_fn(question, history=None):
    if history:
        answer = chat_engine.chat(question, chat_history=[ChatMessage(role=MessageRole.USER if msg["role"] =="user" else MessageRole.ASSISTANT,
                                                          content=msg["content"]) for msg in history])
    else:
        answer = chat_engine.chat(question, chat_history=[])
    return str(answer)

def get_answer_fn(question: str, history=None) -> str:
    """A function representing your RAG agent."""
    # Format appropriately the history for your RAG agent
    messages = history if history else []
    messages.append({"role": "user", "content": question})

    # Get the answer and the documents
    #agent_output = get_answer_from_agent(messages)
    answer = answer_fn(question, history)
    print(f"answer: {answer}")
    retrieved_nodes = retriever.retrieve(question)
    #print(retrieved_nodes)
    documents = [node.node.text for node in retrieved_nodes]
    #print(documents)

    # Following llama_index syntax, you can get the answer and the retrieved documents
    #answer = agent_output.text
    #documents = agent_output.source_nodes

    # Instead of returning a simple string, we return the AgentAnswer object which
    # allows us to specify the retrieved context which is used by RAGAS metrics
    return AgentAnswer(
        message=answer,
        documents=documents
    )
testset = QATestset.load("../giskard_test_sets/LL144_Sample.jsonl")
report = evaluate(get_answer_fn,
                testset=testset,
                knowledge_base=knowledge_base,
                metrics=[ragas_context_recall, ragas_faithfulness, ragas_answer_relevancy, ragas_context_precision])

# Adaptive Method
#report.to_html("simple_rewriter_results.html")
results = report.to_pandas()
results.to_csv('rewriter_with_topics4.csv', index=False)
report.to_html("rewriter_with_topics4.html")

### Evaluate optimal n queries: n=3, n=5, n=7 || k=10 for all

In [ ]:
# SET RESULTS FILENAME:
results_path = 'rewriter_n3_k10'


Settings.llm=llm_gpt35
query_gen_prompt = PromptTemplate(query_gen_prompt_str)

def generate_queries(llm, query_str: str, num_queries: int = 6, topics_in_each_node=topics_in_each_node):
    fmt_prompt = query_gen_prompt.format(
        num_queries=num_queries - 1, query=query_str, topics_in_each_node=topics_in_each_node
    )
    response = llm.complete(fmt_prompt)
    queries = response.text.split("\n")
    return queries

async def run_queries(queries, retrievers):
    """Run queries against retrievers."""
    tasks = []
    for query in queries:
        for i, retriever in enumerate(retrievers):
            tasks.append(retriever.aretrieve(query))

    task_results = await tqdm.gather(*tasks)

    results_dict = {}
    for i, (query, query_result) in enumerate(zip(queries, task_results)):
        results_dict[(query, i)] = query_result
    #print(results_dict)
    return results_dict

## vector retriever
vector_retriever = index.as_retriever(similarity_top_k=10)

## bm25 retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=10
)

def fuse_results(results_dict, similarity_top_k: int = 5):
    """Fuse results."""
    k = 60.0  # `k` is a parameter used to control the impact of outlier rankings.
    fused_scores = {}
    text_to_node = {}

    # compute reciprocal rank scores
    for nodes_with_scores in results_dict.values():
        for rank, node_with_score in enumerate(
            sorted(
                nodes_with_scores, key=lambda x: x.score or 0.0, reverse=True
            )
        ):
            text = node_with_score.node.get_content()
            text_to_node[text] = node_with_score
            if text not in fused_scores:
                fused_scores[text] = 0.0
            fused_scores[text] += 1.0 / (rank + k)

    # sort results
    reranked_results = dict(
        sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    )

    # adjust node scores
    reranked_nodes: List[NodeWithScore] = []
    for text, score in reranked_results.items():
        reranked_nodes.append(text_to_node[text])
        reranked_nodes[-1].score = score

    return reranked_nodes[:similarity_top_k]


class FusionRetriever(BaseRetriever):
    """Ensemble retriever with fusion."""

    def __init__(
        self,
        llm,
        retrievers: List[BaseRetriever],
        similarity_top_k: int = 5,
    ) -> None:
        """Init params."""
        self._retrievers = retrievers
        self._similarity_top_k = similarity_top_k
        self._llm = llm
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        queries = generate_queries(
            self._llm, query_bundle.query_str, num_queries=4, topics_in_each_node=topics_in_each_node
        )

        results = asyncio.run(run_queries(queries, self._retrievers))

        final_results = fuse_results(
            results, similarity_top_k=self._similarity_top_k
        )
        #print(f"final_results {final_results}")

        return final_results
    

# create vector index
text_nodes = splitter(documents)
knowledge_base_df = pd.DataFrame([node.text for node in all_nodes], columns=["text"])
knowledge_base = KnowledgeBase(knowledge_base_df)
retriever = FusionRetriever(
    llm_gpt35, [vector_retriever, bm25_retriever], similarity_top_k=10)


Settings.llm = llm_gpt4o
def answer_fn(question, history=None):
    if history:
        answer = chat_engine.chat(question, chat_history=[ChatMessage(role=MessageRole.USER if msg["role"] =="user" else MessageRole.ASSISTANT,
                                                          content=msg["content"]) for msg in history])
    else:
        answer = chat_engine.chat(question, chat_history=[])
    return str(answer)

def get_answer_fn(question: str, history=None) -> str:
    """A function representing your RAG agent."""
    messages = history if history else []
    messages.append({"role": "user", "content": question})
    answer = answer_fn(question, history)
    retrieved_nodes = retriever.retrieve(question)
    documents = [node.node.text for node in retrieved_nodes]
    # Instead of returning a simple string, we return the AgentAnswer object which
    # allows us to specify the retrieved context which is used by RAGAS metrics
    return AgentAnswer(
        message=answer,
        documents=documents
    )

testset = QATestset.load("../giskard_test_sets/LL144_Sample.jsonl")
report = evaluate(get_answer_fn,
                testset=testset,
                knowledge_base=knowledge_base,
                metrics=[ragas_context_recall, ragas_faithfulness, ragas_answer_relevancy, ragas_context_precision])


results = report.to_pandas()
csv_path = results_path + ".csv"
html_path = results_path + ".html"

results.to_csv(csv_path, index=False)
report.to_html(html_path)

In [ ]:
# SET RESULTS FILENAME:
results_path = 'rewriter_n5_k10'


Settings.llm=llm_gpt35
query_gen_prompt = PromptTemplate(query_gen_prompt_str)

def generate_queries(llm, query_str: str, num_queries: int = 6, topics_in_each_node=topics_in_each_node):
    fmt_prompt = query_gen_prompt.format(
        num_queries=num_queries - 1, query=query_str, topics_in_each_node=topics_in_each_node
    )
    response = llm.complete(fmt_prompt)
    queries = response.text.split("\n")
    return queries

async def run_queries(queries, retrievers):
    """Run queries against retrievers."""
    tasks = []
    for query in queries:
        for i, retriever in enumerate(retrievers):
            tasks.append(retriever.aretrieve(query))

    task_results = await tqdm.gather(*tasks)

    results_dict = {}
    for i, (query, query_result) in enumerate(zip(queries, task_results)):
        results_dict[(query, i)] = query_result
    #print(results_dict)
    return results_dict

## vector retriever
vector_retriever = index.as_retriever(similarity_top_k=10)

## bm25 retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=10
)

def fuse_results(results_dict, similarity_top_k: int = 5):
    """Fuse results."""
    k = 60.0  # `k` is a parameter used to control the impact of outlier rankings.
    fused_scores = {}
    text_to_node = {}

    # compute reciprocal rank scores
    for nodes_with_scores in results_dict.values():
        for rank, node_with_score in enumerate(
            sorted(
                nodes_with_scores, key=lambda x: x.score or 0.0, reverse=True
            )
        ):
            text = node_with_score.node.get_content()
            text_to_node[text] = node_with_score
            if text not in fused_scores:
                fused_scores[text] = 0.0
            fused_scores[text] += 1.0 / (rank + k)

    # sort results
    reranked_results = dict(
        sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    )

    # adjust node scores
    reranked_nodes: List[NodeWithScore] = []
    for text, score in reranked_results.items():
        reranked_nodes.append(text_to_node[text])
        reranked_nodes[-1].score = score

    return reranked_nodes[:similarity_top_k]


class FusionRetriever(BaseRetriever):
    """Ensemble retriever with fusion."""

    def __init__(
        self,
        llm,
        retrievers: List[BaseRetriever],
        similarity_top_k: int = 5,
    ) -> None:
        """Init params."""
        self._retrievers = retrievers
        self._similarity_top_k = similarity_top_k
        self._llm = llm
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        queries = generate_queries(
            self._llm, query_bundle.query_str, num_queries=6, topics_in_each_node=topics_in_each_node
        )

        results = asyncio.run(run_queries(queries, self._retrievers))

        final_results = fuse_results(
            results, similarity_top_k=self._similarity_top_k
        )
        #print(f"final_results {final_results}")

        return final_results
    

# create vector index
text_nodes = splitter(documents)
knowledge_base_df = pd.DataFrame([node.text for node in all_nodes], columns=["text"])
knowledge_base = KnowledgeBase(knowledge_base_df)
retriever = FusionRetriever(
    llm_gpt35, [vector_retriever, bm25_retriever], similarity_top_k=10)


Settings.llm = llm_gpt4o
def answer_fn(question, history=None):
    if history:
        answer = chat_engine.chat(question, chat_history=[ChatMessage(role=MessageRole.USER if msg["role"] =="user" else MessageRole.ASSISTANT,
                                                          content=msg["content"]) for msg in history])
    else:
        answer = chat_engine.chat(question, chat_history=[])
    return str(answer)

def get_answer_fn(question: str, history=None) -> str:
    """A function representing your RAG agent."""
    messages = history if history else []
    messages.append({"role": "user", "content": question})
    answer = answer_fn(question, history)
    retrieved_nodes = retriever.retrieve(question)
    documents = [node.node.text for node in retrieved_nodes]
    # Instead of returning a simple string, we return the AgentAnswer object which
    # allows us to specify the retrieved context which is used by RAGAS metrics
    return AgentAnswer(
        message=answer,
        documents=documents
    )

testset = QATestset.load("../giskard_test_sets/LL144_Sample.jsonl")
report = evaluate(get_answer_fn,
                testset=testset,
                knowledge_base=knowledge_base,
                metrics=[ragas_context_recall, ragas_faithfulness, ragas_answer_relevancy, ragas_context_precision])


results = report.to_pandas()
csv_path = results_path + ".csv"
html_path = results_path + ".html"

results.to_csv(csv_path, index=False)
report.to_html(html_path)

In [ ]:
# SET RESULTS FILENAME:
results_path = 'rewriter_n7_k10'


Settings.llm=llm_gpt35
query_gen_prompt = PromptTemplate(query_gen_prompt_str)

def generate_queries(llm, query_str: str, num_queries: int = 6, topics_in_each_node=topics_in_each_node):
    fmt_prompt = query_gen_prompt.format(
        num_queries=num_queries - 1, query=query_str, topics_in_each_node=topics_in_each_node
    )
    response = llm.complete(fmt_prompt)
    queries = response.text.split("\n")
    return queries

async def run_queries(queries, retrievers):
    """Run queries against retrievers."""
    tasks = []
    for query in queries:
        for i, retriever in enumerate(retrievers):
            tasks.append(retriever.aretrieve(query))

    task_results = await tqdm.gather(*tasks)

    results_dict = {}
    for i, (query, query_result) in enumerate(zip(queries, task_results)):
        results_dict[(query, i)] = query_result
    #print(results_dict)
    return results_dict

## vector retriever
vector_retriever = index.as_retriever(similarity_top_k=10)

## bm25 retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=10
)

def fuse_results(results_dict, similarity_top_k: int = 5):
    """Fuse results."""
    k = 60.0  # `k` is a parameter used to control the impact of outlier rankings.
    fused_scores = {}
    text_to_node = {}

    # compute reciprocal rank scores
    for nodes_with_scores in results_dict.values():
        for rank, node_with_score in enumerate(
            sorted(
                nodes_with_scores, key=lambda x: x.score or 0.0, reverse=True
            )
        ):
            text = node_with_score.node.get_content()
            text_to_node[text] = node_with_score
            if text not in fused_scores:
                fused_scores[text] = 0.0
            fused_scores[text] += 1.0 / (rank + k)

    # sort results
    reranked_results = dict(
        sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    )

    # adjust node scores
    reranked_nodes: List[NodeWithScore] = []
    for text, score in reranked_results.items():
        reranked_nodes.append(text_to_node[text])
        reranked_nodes[-1].score = score

    return reranked_nodes[:similarity_top_k]


class FusionRetriever(BaseRetriever):
    """Ensemble retriever with fusion."""

    def __init__(
        self,
        llm,
        retrievers: List[BaseRetriever],
        similarity_top_k: int = 5,
    ) -> None:
        """Init params."""
        self._retrievers = retrievers
        self._similarity_top_k = similarity_top_k
        self._llm = llm
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        queries = generate_queries(
            self._llm, query_bundle.query_str, num_queries=8, topics_in_each_node=topics_in_each_node
        )

        results = asyncio.run(run_queries(queries, self._retrievers))

        final_results = fuse_results(
            results, similarity_top_k=self._similarity_top_k
        )
        #print(f"final_results {final_results}")

        return final_results
    

# create vector index
text_nodes = splitter(documents)
knowledge_base_df = pd.DataFrame([node.text for node in all_nodes], columns=["text"])
knowledge_base = KnowledgeBase(knowledge_base_df)
retriever = FusionRetriever(
    llm_gpt35, [vector_retriever, bm25_retriever], similarity_top_k=10)


Settings.llm = llm_gpt4o
def answer_fn(question, history=None):
    if history:
        answer = chat_engine.chat(question, chat_history=[ChatMessage(role=MessageRole.USER if msg["role"] =="user" else MessageRole.ASSISTANT,
                                                          content=msg["content"]) for msg in history])
    else:
        answer = chat_engine.chat(question, chat_history=[])
    return str(answer)

def get_answer_fn(question: str, history=None) -> str:
    """A function representing your RAG agent."""
    messages = history if history else []
    messages.append({"role": "user", "content": question})
    answer = answer_fn(question, history)
    
    retrieved_nodes = retriever.retrieve(question)
    documents = [node.node.text for node in retrieved_nodes]
    # Instead of returning a simple string, we return the AgentAnswer object which
    # allows us to specify the retrieved context which is used by RAGAS metrics
    return AgentAnswer(
        message=answer,
        documents=documents
    )

testset = QATestset.load("../giskard_test_sets/LL144_Sample.jsonl")
report = evaluate(get_answer_fn,
                testset=testset,
                knowledge_base=knowledge_base,
                metrics=[ragas_context_recall, ragas_faithfulness, ragas_answer_relevancy, ragas_context_precision])


results = report.to_pandas()
csv_path = results_path + ".csv"
html_path = results_path + ".html"

results.to_csv(csv_path, index=False)
report.to_html(html_path)

### Evaluate top-k, No rewriter

In [ ]:
# SET RESULTS FILENAME BASE:
results_base_path = 'hybrid_search_k5'

Settings.llm = llm_gpt35

async def run_queries(query, retrievers):
    """Run query against retrievers."""
    tasks = []
    for i, retriever in enumerate(retrievers):
        tasks.append(retriever.aretrieve(query))

    task_results = await asyncio.gather(*tasks)

    results_dict = {}
    for i, query_result in enumerate(task_results):
        results_dict[i] = query_result
    return results_dict

## vector retriever
vector_retriever = index.as_retriever(similarity_top_k=10)

## bm25 retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=10
)

def fuse_results(results_dict, similarity_top_k: int = 5):
    """Fuse results."""
    k = 60.0  # `k` is a parameter used to control the impact of outlier rankings.
    fused_scores = {}
    text_to_node = {}

    # compute reciprocal rank scores
    for nodes_with_scores in results_dict.values():
        for rank, node_with_score in enumerate(
            sorted(
                nodes_with_scores, key=lambda x: x.score or 0.0, reverse=True
            )
        ):
            text = node_with_score.node.get_content()
            text_to_node[text] = node_with_score
            if text not in fused_scores:
                fused_scores[text] = 0.0
            fused_scores[text] += 1.0 / (rank + k)

    # sort results
    reranked_results = dict(
        sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    )

    # adjust node scores
    reranked_nodes: List[NodeWithScore] = []
    for text, score in reranked_results.items():
        reranked_nodes.append(text_to_node[text])
        reranked_nodes[-1].score = score

    return reranked_nodes[:similarity_top_k]


class FusionRetriever(BaseRetriever):
    """Ensemble retriever with fusion."""

    def __init__(
        self,
        retrievers: List[BaseRetriever],
        similarity_top_k: int = 5,
    ) -> None:
        """Init params."""
        self._retrievers = retrievers
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        results = asyncio.run(run_queries(query_bundle.query_str, self._retrievers))

        final_results = fuse_results(
            results, similarity_top_k=self._similarity_top_k
        )

        return final_results


splitter = SentenceSplitter(chunk_size=512)
index = VectorStoreIndex.from_documents(documents, transformations=[splitter])
chat_engine = index.as_chat_engine()
# create vector index
text_nodes = splitter(documents)
knowledge_base_df = pd.DataFrame([node.text for node in all_nodes], columns=["text"])
knowledge_base = KnowledgeBase(knowledge_base_df)

Settings.llm = llm_gpt4o

def answer_fn(question, history=None):
    if history:
        answer = chat_engine.chat(question, chat_history=[ChatMessage(role=MessageRole.USER if msg["role"] == "user" else MessageRole.ASSISTANT, content=msg["content"]) for msg in history])
    else:
        answer = chat_engine.chat(question, chat_history=[])
    return str(answer)

def get_answer_fn(question: str, history=None) -> str:
    """A function representing your RAG agent."""
    messages = history if history else []
    messages.append({"role": "user", "content": question})
    answer = answer_fn(question, history)
    print(f"question: {question}")
    print(f"answer: {answer}")
    retrieved_nodes = retriever.retrieve(question)
    documents = [node.node.text for node in retrieved_nodes]
    return AgentAnswer(
        message=answer,
        documents=documents
    )

testset = QATestset.load("../giskard_test_sets/LL144_275_New.jsonl")

for k in [3, 5, 7, 10]:
    results_path = f'{results_base_path}{k}'
    
    retriever = FusionRetriever([vector_retriever, bm25_retriever], similarity_top_k=k)
    
    report = evaluate(get_answer_fn,
                    testset=testset,
                    knowledge_base=knowledge_base,
                    metrics=[ragas_context_recall, ragas_faithfulness, ragas_answer_relevancy, ragas_context_precision])

    results = report.to_pandas()
    csv_path = results_path + ".csv"
    html_path = results_path + ".html"

    results.to_csv(csv_path, index=False)
    report.to_html(html_path)


In [ ]:
print(text_nodes)

In [ ]:
from typing import List

from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore
import asyncio

# Reciprocal rank fusion
from typing import List
from llama_index.core.schema import NodeWithScore

# Vector Search on Each QUery

from tqdm.asyncio import tqdm

# Query Gen/Rewriting stage
from llama_index.core import PromptTemplate
import time


import os
from llama_index.llms.groq import Groq
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.azure_openai import AzureOpenAI
from dotenv import load_dotenv

import nest_asyncio

nest_asyncio.apply()

from pathlib import Path
from llama_index.readers.file import PyMuPDFReader

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5")

load_dotenv()
AZURE_API_KEY = os.getenv('AZURE_API_KEY')
AZURE_DEPLOYMENT_NAME = os.getenv("AZURE_DEPLOYMENT_NAME")
AZURE_API_VERSION = os.getenv("AZURE_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")

# setup LLM
import giskard
from llama_index.llms.azure_openai import AzureOpenAI
from giskard.llm import set_llm_model, set_llm_api
from giskard.llm.client import get_default_client
from giskard.llm.client import set_llm_api, set_llm_model
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.readers.file import PyMuPDFReader
from llama_index.core.base.llms.types import ChatMessage, MessageRole
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
import os
from giskard.rag import KnowledgeBase, generate_testset, QATestset


llm_gpt4o = AzureOpenAI(
    deployment_name="gpt4o",
    temperature=0, 
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_API_VERSION
)

llm_gpt35 = AzureOpenAI(
    deployment_name=AZURE_DEPLOYMENT_NAME,
    temperature=0, 
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_API_VERSION
)

embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-large-en-v1.5"
)

Settings.llm = llm_gpt4o
Settings.embed_model = embed_model


from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter


loader = PyMuPDFReader()
documents2 = loader.load(file_path="../legal_data//LL144/LL144_Definitions.pdf")
documents1 = loader.load(file_path="../legal_data//LL144/LL144.pdf")
documents = documents1 + documents2

splitter = SentenceSplitter(chunk_size=512)
#index = VectorStoreIndex.from_documents(
#    documents, transformations=[splitter], embed_model=embed_model, llm=llm_gpt35
#)



query_str = "What is a bias audit and why do I need to do one?"


legal_prompt_str = (
    "You are an expert legal assistant that generates multiple search queries based on a "
    "single input query about a particular set of regulation. Your aim is to gather information"
    "that would contribute to a complete answer to the single query, tailored to legal clients."
    "Consider (if applicable) the following:"
    "- definitions"
    "- requirements"
    "- scope"
    "- jurisdiction"
    "- penalties"
    "Generate {num_queries} search queries, one on each line, "
    "related to the following iput query:\n"
    "Query: {query}\n"
    "Queries:\n"
)

legal_prompt_str = """
You are an expert legal assistant that generates multiple search queries based on a 
single input query about a particular set of regulation. Your aim is to gather information
that would contribute to a complete answer to the single query, tailored to legal clients.

Use any relevant keywords from the following list to improve the specificity of your queries:

- New York City Department of Consumer and Worker Protection (DCWP)
- Commissioner of the DCWP
- New York City Charter

Bias Audit Requirements
- Clarifications and Definitions
- Public Feedback and Revisions
- Impact Ratio Calculation
- Data for Bias Audits
- Shared Bias Audits

Automated Employment Decision Tool (AEDT)
- Bias Audit
- Candidate for Employment
- Employers
- Employment Agencies
- EEOC
- Independent Auditors
- Bias Audit
- Impact Ratio
- Machine Learning, Statistical Modeling, Data Analytics, or Artificial Intelligence

Scoring Rate
- Screen
- Selection Rate
- Definition of Test Data
- Bias Audit Requirement
- Audit Calculations
- Group Classification
- Exclusion of Unknown Categories
- Example Scenario

Employers
- Employment Agencies
- AEDTs
- Vendors Conducting Bias Audits
- Use of AEDT in Hiring Processes
- Requirement for Bias Audits
- Example of Bias Audit Process
- Metrics for Evaluating Bias (sex categories, number of applicants, number selected, selection rate, impact ratio)

Employer
- AEDT (Automated Employment Decision Tool)
- Vendor
- Independent Auditor
- Applicants (Male and Female)

Race/Ethnicity Categories
- Hispanic or Latino
- White (Not Hispanic or Latino)
- Black or African American (Not Hispanic or Latino)

Intersectional Categories
- Hispanic or Latino
- Non-Hispanic or Latino
- Male

Automated Employment Decision Tool (AEDT)
- Candidates for Employment
- Employees Being Considered for Promotion

Employers
- Independent Auditors
- Demographic Groups (sex, race/ethnicity, intersectional categories)
- Independent Auditor
- Employer
- Automated Employment Decision Tools (AEDTs)

Sex Categories
- Male
- Female

Race/Ethnicity Categories
- Hispanic or Latino
- White (Not Hispanic or Latino)
- Black or African American (Not Hispanic or Latino)
- Native Hawaiian or Pacific Islander (Not Hispanic or Latino)
- Asian (Not Hispanic or Latino)
- Native American or Alaska Native (Not Hispanic or Latino)
- Two or More Races (Not Hispanic or Latino)

Intersectional Categories
- Data Exclusion
- Historical Data Requirements
- Requirements for Conducting a Bias Audit

Automated Employment Decision Tool (AEDT)
- Use of Historical Data and Test Data
- Employment Agencies
- Employers
- AEDTs (Automated Employment Decision Tools)

Bias Audit Restrictions
- Published Results (§ 5-303)
- Notice to Candidates and Employees (§ 5-304)
- Automated Employment Decision Tool (AEDT)
- Employers and Employment Agencies
- Candidates for Employment

Employers
- Employment Agencies
- Candidates for Employment
- New York City Council
- Council Members
- Administrative Code of the City of New York

Bias Audit Requirement
- Notification Requirements
- Legal References
- Disclosure Requirements
- Penalties
- Enforcement

Office of Administrative Trials and Hearings
- City Agencies
- Corporation Counsel
- File #: Int 1894-2020
- New York City Commission on Human Rights
- Title 8


Generate {num_queries} search queries, one on each line, 
related to the following input query:\n
Query: {query}\n
Queries:\n


"""

query_gen_prompt_str = legal_prompt_str



query_gen_prompt = PromptTemplate(query_gen_prompt_str)

def generate_queries(llm, query_str: str, num_queries: int = 6):
    fmt_prompt = query_gen_prompt.format(
        num_queries=num_queries - 1, query=query_str
    )
    response = llm.complete(fmt_prompt)
    queries = response.text.split("\n")
    print(queries)
    return queries


async def run_queries(queries, retrievers):
    """Run queries against retrievers."""
    tasks = []
    for query in queries:
        for i, retriever in enumerate(retrievers):
            tasks.append(retriever.aretrieve(query))

    task_results = await tqdm.gather(*tasks)

    results_dict = {}
    for i, (query, query_result) in enumerate(zip(queries, task_results)):
        results_dict[(query, i)] = query_result

    return results_dict



# get retrievers
from llama_index.retrievers.bm25 import BM25Retriever


## vector retriever
vector_retriever = index.as_retriever(similarity_top_k=5)

## bm25 retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=5
)

def fuse_results(results_dict, similarity_top_k: int = 5):
    """Fuse results."""
    k = 60.0  # `k` is a parameter used to control the impact of outlier rankings.
    fused_scores = {}
    text_to_node = {}

    # compute reciprocal rank scores
    for nodes_with_scores in results_dict.values():
        for rank, node_with_score in enumerate(
            sorted(
                nodes_with_scores, key=lambda x: x.score or 0.0, reverse=True
            )
        ):
            text = node_with_score.node.get_content()
            text_to_node[text] = node_with_score
            if text not in fused_scores:
                fused_scores[text] = 0.0
            fused_scores[text] += 1.0 / (rank + k)

    # sort results
    reranked_results = dict(
        sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    )

    # adjust node scores
    reranked_nodes: List[NodeWithScore] = []
    for text, score in reranked_results.items():
        reranked_nodes.append(text_to_node[text])
        reranked_nodes[-1].score = score

    return reranked_nodes[:similarity_top_k]



class FusionRetriever(BaseRetriever):
    """Ensemble retriever with fusion."""

    def __init__(
        self,
        llm,
        retrievers: List[BaseRetriever],
        similarity_top_k: int = 5,
    ) -> None:
        """Init params."""
        self._retrievers = retrievers
        self._similarity_top_k = similarity_top_k
        self._llm = llm
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        queries = generate_queries(
            self._llm, query_bundle.query_str, num_queries=4
        )
        results = asyncio.run(run_queries(queries, self._retrievers))
        final_results = fuse_results(
            results, similarity_top_k=self._similarity_top_k
        )

        return final_results
    

from llama_index.core.query_engine import RetrieverQueryEngine

splitter = SentenceSplitter(chunk_size=512)
#index = VectorStoreIndex.from_documents(documents, transformations=[splitter])
chat_engine = index.as_chat_engine()
# create vector index
text_nodes = splitter(documents)
knowledge_base_df = pd.DataFrame([node.text for node in all_nodes], columns=["text"])
knowledge_base = KnowledgeBase(knowledge_base_df)
retriever = FusionRetriever(
    llm_gpt35, [vector_retriever, bm25_retriever], similarity_top_k=5)


Settings.llm = llm_gpt35
def answer_fn(question, history=None):
    if history:
        answer = chat_engine.chat(question, chat_history=[ChatMessage(role=MessageRole.USER if msg["role"] =="user" else MessageRole.ASSISTANT,
                                                          content=msg["content"]) for msg in history])
    else:
        answer = chat_engine.chat(question, chat_history=[])
    return str(answer)

def get_answer_fn(question: str, history=None) -> str:
    """A function representing your RAG agent."""
    messages = history if history else []
    messages.append({"role": "user", "content": question})
    answer = answer_fn(question, history)
    print(answer)
    retrieved_nodes = retriever.retrieve(question)
    documents = [node.node.text for node in retrieved_nodes]
    # Instead of returning a simple string, we return the AgentAnswer object which
    # allows us to specify the retrieved context which is used by RAGAS metrics
    return AgentAnswer(
        message=answer,
        documents=documents
    )

testset = QATestset.load("../giskard_test_sets/LL144_Sample.jsonl")
report = evaluate(get_answer_fn,
                testset=testset,
                knowledge_base=knowledge_base,
                metrics=[ragas_context_recall, ragas_faithfulness, ragas_answer_relevancy, ragas_context_precision])


results = report.to_pandas()
csv_path = results_path + ".csv"
html_path = results_path + ".html"

results.to_csv(csv_path, index=False)
report.to_html(html_path)

In [ ]:
retriever = FusionRetriever(
    llm_gpt35, [vector_retriever, bm25_retriever], similarity_top_k=10)


retrieved_nodes = retriever.retrieve("what is a bias audit?")

print(retrieved_nodes)

In [ ]:
for node in retrieved_nodes:
    print(node.get_content())

#### Evaluate query router method w/ adaptive k value

In [ ]:
import nest_asyncio

nest_asyncio.apply()

from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
)
from llama_index.core.evaluation import (
    DatasetGenerator,
    FaithfulnessEvaluator,
    RelevancyEvaluator
)
from llama_index.llms.azure_openai import AzureOpenAI

import openai
import time

# Load environment variables from .env file
import pandas as pd
from llama_index.core.evaluation import (
    RetrieverEvaluator,
    get_retrieval_results_df,
)
import os
import logging
import pandas as pd
from dotenv import load_dotenv
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.readers.file import PyMuPDFReader
from llama_index.core.node_parser import SentenceSplitter
from datetime import datetime
from llama_index.core import (
    StorageContext, VectorStoreIndex, SimpleDirectoryReader, 
    get_response_synthesizer, Settings
)
import traceback
from llama_index.core.evaluation import (
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset,
)
from llama_index.core.evaluation import generate_question_context_pairs, QueryResponseDataset
from llama_index.llms.openai import OpenAI

from typing import List
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore
import asyncio
from tqdm.asyncio import tqdm
from llama_index.core import PromptTemplate
import time
from llama_index.retrievers.bm25 import BM25Retriever


from pathlib import Path
from llama_index.readers.file import PDFReader
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
import json
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import IndexNode
from llama_index.core.embeddings import resolve_embed_model
from llama_index.core import Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import IndexNode
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
)

import giskard
from llama_index.llms.azure_openai import AzureOpenAI
from giskard.llm import set_llm_model, set_llm_api
from giskard.llm.client import get_default_client
from giskard.llm.client import set_llm_api, set_llm_model
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.readers.file import PyMuPDFReader
from llama_index.core.base.llms.types import ChatMessage, MessageRole
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
import os
from giskard.rag import KnowledgeBase, generate_testset, QATestset
#from giskard.rag.question_generators import complex_questions, complex_situational_questions, compare_questions, simple_questions, rule_conclusion_questions, distracting_questions, na_questions, vague_questions, oos_questions, situational_questions, double_questions, conversational_questions
from giskard.rag import AgentAnswer
from giskard.rag import evaluate, RAGReport
from giskard.rag.metrics.ragas_metrics import ragas_context_recall, ragas_faithfulness, ragas_answer_relevancy, ragas_context_precision
from llama_index.core.base.llms.types import ChatMessage, MessageRole
from llama_index.core.query_engine import RetrieverQueryEngine


In [ ]:
import os
import sys
import logging
import traceback
import pandas as pd
import nest_asyncio
import asyncio
from datetime import datetime
from dotenv import load_dotenv
from typing import List
from huggingface_hub import InferenceApi
from transformers import pipeline
from IPython.display import Markdown, display

nest_asyncio.apply()
load_dotenv()

# LLamaIndex Imports
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.vector_stores.neo4jvector import Neo4jVectorStore
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings, StorageContext, get_response_synthesizer,PropertyGraphIndex, Document, KnowledgeGraphIndex
from llama_index.core.evaluation import (DatasetGenerator,FaithfulnessEvaluator,RelevancyEvaluator)
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.readers.file import PyMuPDFReader
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
from llama_index.core.schema import IndexNode, NodeWithScore, Document, QueryBundle
from llama_index.core.extractors import (SummaryExtractor,QuestionsAnsweredExtractor)
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.retrievers import BaseRetriever, VectorIndexRetriever, KGTableRetriever
from llama_index.core.query_engine import RetrieverQueryEngine, KnowledgeGraphQueryEngine, CitationQueryEngine
from llama_index.core.prompts.base import PromptTemplate, PromptType
from llama_index.graph_stores.neo4j import Neo4jGraphStore
from llama_index.core.base.llms.types import ChatMessage, MessageRole
from llama_index.core.retrievers import BaseRetriever


# Giskard imports
import giskard
from giskard.rag import AgentAnswer, evaluate, RAGReport, KnowledgeBase, generate_testset, QATestset
from giskard.rag.metrics.ragas_metrics import ragas_context_recall, ragas_faithfulness, ragas_answer_relevancy, ragas_context_precision
from giskard.llm import set_llm_model, set_llm_api
from giskard.llm.client import get_default_client
from giskard.llm.embeddings import set_default_embedding, get_default_embedding

def remove_openai_api_key():
    if "OPENAI_API_KEY" in os.environ:
        del os.environ["OPENAI_API_KEY"]

# Need to specify this here otherwise it doesn't work - Giskard Problem (?)
os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("GSK_AZURE_OPENAI_API_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("GSK_AZURE_OPENAI_ENDPOINT")
os.environ["AZURE_API_VERSION"] = os.getenv("AZURE_API_VERSION")
os.environ["GSK_LLM_API"] = "azure"
os.environ["GSK_LLM_MODEL"] = "gpt-4o-mini"
set_llm_api("azure")
set_llm_model('gpt-4o-mini')

AZURE_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
AZURE_DEPLOYMENT_NAME = os.getenv("AZURE_DEPLOYMENT_NAME")
AZURE_API_VERSION = os.getenv("AZURE_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")


embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5")
Settings.embed_model = embed_model

# Setup LLM
llm_gpt4o = AzureOpenAI(
    deployment_name="gpt-4o-mini",
    temperature=0, 
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_API_VERSION
)

llm_gpt35 = AzureOpenAI(
    deployment_name="gpt35",
    temperature=0, 
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_API_VERSION
)

# Setup LLM
llm_gpt4o_ = AzureOpenAI(
    deployment_name="gpt4o",
    temperature=0,
    api_key=os.getenv("GPT4O_API_KEY"),
    azure_endpoint=os.getenv("GPT4O_AZURE_ENDPOINT"),
    api_version=os.getenv("GPT4O_API_VERSION")
)

Settings.llm = llm_gpt35

# Verify LLM setup
client = get_default_client()

#print("Client base URL:", client._client._base_url)
#print("Client API key:", client._client.api_key)
#print("Client API version:", client._client._api_version)
#print("Client model:", client.model)

assert client._client._base_url == f'{os.environ["AZURE_OPENAI_ENDPOINT"]}/openai/'
assert client._client.api_key == os.environ["AZURE_OPENAI_API_KEY"]
assert client._client._api_version == os.environ["OPENAI_API_VERSION"]

url = os.getenv("NEO4J_URI")
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")
database = os.getenv("NEO4J_DATABASE")

# Load documents
loader = PyMuPDFReader()
documents2 = loader.load(file_path='../legal_data//LL144/LL144_Definitions.pdf')
documents1 = loader.load(file_path='../legal_data//LL144/LL144.pdf')
documents = documents1 + documents2

# Create vector index
splitter = SentenceSplitter(chunk_size=512)
index = VectorStoreIndex.from_documents(documents, transformations=[splitter])
chat_engine = index.as_chat_engine(chat_mode='context')

text_nodes = splitter(documents)
knowledge_base_df = pd.DataFrame([node.text for node in text_nodes], columns=['text'])
knowledge_base = KnowledgeBase(knowledge_base_df)

## vector retriever
vector_retriever = index.as_retriever(similarity_top_k=10)

## bm25 retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=10
)

# Set results filename base
results_base_path = 'hybrid_search2_k'

# Define async run queries
async def run_queries(query, retrievers):
    tasks = [retriever.aretrieve(query) for retriever in retrievers]
    task_results = await asyncio.gather(*tasks)
    return {i: result for i, result in enumerate(task_results)}

# Define fusion retriever
def fuse_results(results_dict, similarity_top_k=5):
    k = 60.0
    fused_scores = {}
    text_to_node = {}
    for nodes_with_scores in results_dict.values():
        for rank, node_with_score in enumerate(sorted(nodes_with_scores, key=lambda x: x.score or 0.0, reverse=True)):
            text = node_with_score.node.get_content()
            text_to_node[text] = node_with_score
            if text not in fused_scores:
                fused_scores[text] = 0.0
            fused_scores[text] += 1.0 / (rank + k)
    reranked_results = dict(sorted(fused_scores.items(), key=lambda x: x[1], reverse=True))
    reranked_nodes = [text_to_node[text] for text in reranked_results.keys()]
    for text, score in reranked_results.items():
        text_to_node[text].score = score
    return reranked_nodes[:similarity_top_k]

class FusionRetriever(BaseRetriever):
    def __init__(self, retrievers: List[BaseRetriever], similarity_top_k=5):
        self._retrievers = retrievers
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        results = asyncio.run(run_queries(query_bundle.query_str, self._retrievers))
        return fuse_results(results, similarity_top_k=self._similarity_top_k)

Settings.llm = llm_gpt35

def answer_fn(question, history=None):
    chat_history = [ChatMessage(role=MessageRole.USER if msg['role'] == 'user' else MessageRole.ASSISTANT, content=msg['content']) for msg in history] if history else []
    return str(chat_engine.chat(question, chat_history=chat_history))

def get_answer_fn(question: str, history=None) -> str:
    messages = history if history else []
    messages.append({'role': 'user', 'content': question})
    answer = answer_fn(question, history)
    retrieved_nodes = retriever.retrieve(question)
    documents = [node.node.text for node in retrieved_nodes]
    return AgentAnswer(message=answer, documents=documents)

testset = QATestset.load('../giskard_test_sets/LL144_275_New.jsonl')

for k in [3, 5, 7, 10]:
    results_path = f'{results_base_path}{k}'
    retriever = FusionRetriever([vector_retriever, bm25_retriever], similarity_top_k=k)
    report = evaluate(get_answer_fn, testset=testset, knowledge_base=knowledge_base, metrics=[ragas_context_recall, ragas_faithfulness, ragas_answer_relevancy, ragas_context_precision])
    results = report.to_pandas()
    csv_path = results_path + '.csv'
    html_path = results_path + '.html'
    results.to_csv(csv_path, index=False)
    # report.to_html(html_path)



In [ ]:
load_dotenv()
# Fetch API keys from environment variables

AZURE_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
#AZURE_DEPLOYMENT_NAME = os.getenv("AZURE_DEPLOYMENT_NAME")
AZURE_API_VERSION = os.getenv("AZURE_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")


print(AZURE_API_KEY)
#print(AZURE_DEPLOYMENT_NAME)
print(AZURE_API_VERSION)
print(AZURE_OPENAI_ENDPOINT)


from huggingface_hub import InferenceApi
from transformers import pipeline

embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-large-en-v1.5"
)

Settings.embed_model=embed_model

# setup LLM

llm_gpt4o = AzureOpenAI(
    deployment_name="gpt-4o-mini",
    temperature=0, 
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_API_VERSION
)

llm_gpt35 = AzureOpenAI(
    deployment_name="gpt35",#AZURE_DEPLOYMENT_NAME,
    temperature=0, 
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_API_VERSION
)

embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-large-en-v1.5"
)
Settings.embed_model = embed_model

set_llm_api("azure")
set_llm_model('gpt-4o-mini')
# You'll need to provide the name of the model that you've deployed
# Beware, the model provided must be capable of using function calls
client = get_default_client()
assert client._client._base_url == f'{os.environ["AZURE_OPENAI_ENDPOINT"]}/openai/'
assert client._client.api_key == os.environ["AZURE_OPENAI_API_KEY"]
assert client._client._api_version == os.environ["OPENAI_API_VERSION"]
assert client.model == "gpt-4o-mini"#os.environ["AZURE_OPENAI_CHATGPT_DEPLOYMENT"]


# Load documents
loader = PyMuPDFReader()
documents2 = loader.load(file_path='../legal_data//LL144/LL144_Definitions.pdf')
documents1 = loader.load(file_path='../legal_data//LL144/LL144.pdf')
documents = documents1 + documents2

# Create vector index
splitter = SentenceSplitter(chunk_size=512)
index = VectorStoreIndex.from_documents(documents, transformations=[splitter])
chat_engine = index.as_chat_engine(chat_mode='context')

text_nodes = splitter(documents)
knowledge_base_df = pd.DataFrame([node.text for node in text_nodes], columns=['text'])
knowledge_base = KnowledgeBase(knowledge_base_df)

## vector retriever
vector_retriever = index.as_retriever(similarity_top_k=10)

## bm25 retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=10
)

# Set results filename base
results_base_path = 'adaptive_k_ft_classifier_3'

# Load classifier
classifier = pipeline("text-classification", model="rk68/distilbert-q-classifier-3", device='mps')

# Function to classify query and determine top-k value
def classify_query_and_get_topk(query):
    classification = classifier(query)[0]
    label = int(classification['label'].split('_')[-1])
    if label == 0:
        return 3
    elif label == 1:
        return 5
    else:
        return 7

# Async run queries
async def run_queries(query, retrievers):
    tasks = [retriever.aretrieve(query) for retriever in retrievers]
    task_results = await asyncio.gather(*tasks)
    return {i: result for i, result in enumerate(task_results)}

# Define fusion retriever
def fuse_results(results_dict, similarity_top_k):
    k = 60.0
    fused_scores = {}
    text_to_node = {}
    for nodes_with_scores in results_dict.values():
        for rank, node_with_score in enumerate(sorted(nodes_with_scores, key=lambda x: x.score or 0.0, reverse=True)):
            text = node_with_score.node.get_content()
            text_to_node[text] = node_with_score
            if text not in fused_scores:
                fused_scores[text] = 0.0
            fused_scores[text] += 1.0 / (rank + k)
    reranked_results = dict(sorted(fused_scores.items(), key=lambda x: x[1], reverse=True))
    reranked_nodes = [text_to_node[text] for text in reranked_results.keys()]
    for text, score in reranked_results.items():
        text_to_node[text].score = score
    return reranked_nodes[:similarity_top_k]

class FusionRetriever(BaseRetriever):
    def __init__(self, retrievers: List[BaseRetriever]):
        self._retrievers = retrievers
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        top_k = classify_query_and_get_topk(query_bundle.query_str)
        print(f"top-k: {top_k}")
        results = asyncio.run(run_queries(query_bundle.query_str, self._retrievers))
        return fuse_results(results, similarity_top_k=top_k)

def answer_fn(question, history=None):
    chat_history = [ChatMessage(role=MessageRole.USER if msg['role'] == 'user' else MessageRole.ASSISTANT, content=msg['content']) for msg in history] if history else []
    return str(chat_engine.chat(question, chat_history=chat_history))

def get_answer_fn(question: str, history=None) -> str:
    messages = history if history else []
    messages.append({'role': 'user', 'content': question})
    answer = answer_fn(question, history)
    print(f"answer: {answer}")
    retriever = FusionRetriever([vector_retriever, bm25_retriever])
    retrieved_nodes = retriever.retrieve(question)
    print(f"retrieved nodes {retrieved_nodes}")
    documents = [node.node.text for node in retrieved_nodes]
    return AgentAnswer(message=answer, documents=documents)

# Load test set
testset = QATestset.load('../giskard_test_sets/LL144_Sample.jsonl')

results_path = f'{results_base_path}'
report = evaluate(get_answer_fn, testset=testset, knowledge_base=knowledge_base, metrics=[ragas_context_recall, ragas_faithfulness, ragas_answer_relevancy, ragas_context_precision])
results = report.to_pandas()
csv_path = results_path + '.csv'
html_path = results_path + '.html'
results.to_csv(csv_path, index=False)


In [ ]:


# Load documents
loader = PyMuPDFReader()
documents2 = loader.load(file_path='../legal_data//LL144/LL144_Definitions.pdf')
documents1 = loader.load(file_path='../legal_data//LL144/LL144.pdf')
documents = documents1 + documents2

# Create vector index
splitter = SentenceSplitter(chunk_size=512)
index = VectorStoreIndex.from_documents(documents, transformations=[splitter])
chat_engine = index.as_chat_engine(chat_mode='context')

text_nodes = splitter(documents)
knowledge_base_df = pd.DataFrame([node.text for node in text_nodes], columns=['text'])
knowledge_base = KnowledgeBase(knowledge_base_df)

# Vector retriever
vector_retriever = index.as_retriever(similarity_top_k=10)

# BM25 retriever
bm25_retriever = BM25Retriever.from_defaults(docstore=index.docstore, similarity_top_k=10)

# Set results filename base
results_base_path = 'adaptive_k_ft_classifier_2'

# Load classifier
classifier = pipeline("text-classification", model="rk68/distilbert-q-classifier-2", device='mps')

# Function to classify query and determine top-k value
def classify_query_and_get_topk(query):
    classification = classifier(query)[0]
    label = int(classification['label'].split('_')[-1])
    if label == 0:
        return 5
    elif label == 1:
        return 7
    #else:
    #    return 7

# Async run queries
async def run_queries(query, retrievers):
    tasks = [retriever.aretrieve(query) for retriever in retrievers]
    task_results = await asyncio.gather(*tasks)
    return {i: result for i, result in enumerate(task_results)}

# Define fusion retriever
def fuse_results(results_dict, similarity_top_k):
    k = 60.0
    fused_scores = {}
    text_to_node = {}
    for nodes_with_scores in results_dict.values():
        for rank, node_with_score in enumerate(sorted(nodes_with_scores, key=lambda x: x.score or 0.0, reverse=True)):
            text = node_with_score.node.get_content()
            text_to_node[text] = node_with_score
            if text not in fused_scores:
                fused_scores[text] = 0.0
            fused_scores[text] += 1.0 / (rank + k)
    reranked_results = dict(sorted(fused_scores.items(), key=lambda x: x[1], reverse=True))
    reranked_nodes = [text_to_node[text] for text in reranked_results.keys()]
    for text, score in reranked_results.items():
        text_to_node[text].score = score
    return reranked_nodes[:similarity_top_k]

class FusionRetriever(BaseRetriever):
    def __init__(self, retrievers: List[BaseRetriever]):
        self._retrievers = retrievers
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        top_k = classify_query_and_get_topk(query_bundle.query_str)
        #print(f"top-k: {top_k}")
        results = asyncio.run(run_queries(query_bundle.query_str, self._retrievers))
        return fuse_results(results, similarity_top_k=top_k)

def answer_fn(question, history=None):
    chat_history = [ChatMessage(role=MessageRole.USER if msg['role'] == 'user' else MessageRole.ASSISTANT, content=msg['content']) for msg in history] if history else []
    return str(chat_engine.chat(question, chat_history=chat_history))

def get_answer_fn(question: str, history=None) -> str:
    messages = history if history else []
    messages.append({'role': 'user', 'content': question})
    answer = answer_fn(question, history)
    #print(f"answer: {answer}")
    retriever = FusionRetriever([vector_retriever, bm25_retriever])
    retrieved_nodes = retriever.retrieve(question)
    #print(f"retrieved nodes {retrieved_nodes}")
    documents = [node.node.text for node in retrieved_nodes]
    return AgentAnswer(message=answer, documents=documents)

# Load test set
testset = QATestset.load('../giskard_test_sets/LL144_275_New.jsonl')

results_path = f'{results_base_path}'
report = evaluate(get_answer_fn, testset=testset, knowledge_base=knowledge_base, metrics=[ragas_context_recall, ragas_faithfulness, ragas_answer_relevancy, ragas_context_precision])
results = report.to_pandas()
csv_path = results_path + '.csv'
html_path = results_path + '.html'
results.to_csv(csv_path, index=False)


In [ ]:
import json
import csv
from transformers import pipeline
from collections import Counter

# Load the classifier
classifier = pipeline("text-classification", model="rk68/distilbert-q-classifier-3", device='mps')

# Path to the input JSONL file and output CSV file
input_jsonl_path = '../giskard_test_sets/LL144_275_New.jsonl'
output_csv_path = 'output_file.csv'

# Initialize a list to store classification results
classification_results = []

# Read and classify each query in the JSONL file
with open(input_jsonl_path, 'r') as jsonl_file:
    for line in jsonl_file:
        data = json.loads(line)
        query = data['question']  # Use the 'question' field
        classification = classifier(query)[0]
        classification_results.append({'question': query, 'label': classification['label']})

# Write the classification results to a CSV file
with open(output_csv_path, 'w', newline='') as csv_file:
    fieldnames = ['question', 'label']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for result in classification_results:
        writer.writerow(result)

# Calculate and print the percentage of each class label
labels = [result['label'] for result in classification_results]
label_counts = Counter(labels)
total_labels = len(labels)

print("Percentage of each class label:")
for label, count in label_counts.items():
    percentage = (count / total_labels) * 100
    print(f"{label}: {percentage:.2f}%")

In [ ]:
import json
import csv
from transformers import pipeline
from collections import Counter

# Load the classifier
classifier = pipeline("text-classification", model="rk68/distilbert-q-classifier-2", device='mps')

# Path to the input JSONL file and output CSV file
input_jsonl_path = '../giskard_test_sets/LL144_275_New.jsonl'
output_csv_path = 'output_file.csv'

# Initialize a list to store classification results
classification_results = []

# Read and classify each query in the JSONL file
with open(input_jsonl_path, 'r') as jsonl_file:
    for line in jsonl_file:
        data = json.loads(line)
        query = data['question']  # Use the 'question' field
        classification = classifier(query)[0]
        classification_results.append({'question': query, 'label': classification['label']})

# Write the classification results to a CSV file
with open(output_csv_path, 'w', newline='') as csv_file:
    fieldnames = ['question', 'label']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for result in classification_results:
        writer.writerow(result)

# Calculate and print the percentage of each class label
labels = [result['label'] for result in classification_results]
label_counts = Counter(labels)
total_labels = len(labels)

print("Percentage of each class label:")
for label, count in label_counts.items():
    percentage = (count / total_labels) * 100
    print(f"{label}: {percentage:.2f}%")

### GraphRAG Implementation